In [1]:
import hashlib
import hmac
import csv
import numpy as np
import pandas as pd
import joblib
from joblib import Parallel, delayed
from multiprocessing import  Pool
import multiprocessing
import itertools
import random

In [2]:
import math

In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score, precision_recall_curve
from scipy import stats
from collections import Counter

In [4]:
from operator import itemgetter
import logging, sys

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
_DEBUG = False
KAGGLE = False

In [7]:
# logging.basicConfig( #stream=sys.stdout, 
#                     level=logging.DEBUG)

In [8]:
# logging.debug('A debug message!')

In [9]:
pd.options.display.max_rows=2000
pd.options.display.max_columns=500
pd.set_option('display.max_colwidth', None)

In [10]:
import time
from multiprocessing import Value
from ctypes import py_object

In [11]:
def init_pool_processes(shared_value):
    global p
    p = shared_value
def parallelize_dataframe(df, func, data_to_share=None,other_args=None, n_cores=4):
    start = time.time()
    df_split = np.array_split(df, n_cores)
    
    if data_to_share is not None:
        p = Value(py_object)
        p.value = data_to_share        
        pool = Pool(processes=n_cores,
                    initializer=init_pool_processes, initargs=(p,))
    else:
        pool = Pool(n_cores)
        
    arg_list=[]
    arg_list.append(df_split)
            
    if other_args is not None:
        for arg in other_args:
            arg_list.append(itertools.repeat(arg))
    
    parallel_args = zip(*arg_list)
    end = time.time()
    print(f'Parallel preparation time: {end-start}')
    df = pd.concat(pool.starmap(func, parallel_args))
    pool.close()
    pool.join()
    return df

In [12]:
def debugprint(msg):
    if _DEBUG:
        print(msg)

In [13]:
def get_bin(bins,val):
    matching_bin=-1
    for i in range(len(bins)-1):
        if bins[i] <= val < bins[i+1]:
            matching_bin= bins[i]
    if matching_bin==-1:
        matching_bin=bins[-1]
    return matching_bin

In [14]:
def calculate_roll(server_seed,client_seed,nonce):
    # Compute the HMAC-SHA512 hash of string1 using string2 as the key
    
    string1 = f"{nonce}:{server_seed}:{nonce}"
    hmac_key =   f"{nonce}:{client_seed}:{nonce}".encode()
    hmac_hash = hmac.new(key=hmac_key,
                         msg=string1.encode(),
                         digestmod=hashlib.sha512).hexdigest()

    # Convert the first 8 characters of the HMAC-SHA512 hash to an integer
    string3 = hmac_hash[:8]
    number = int(string3, 16)

    # Compute the roll value
    roll = round(number / 429496.7295)
    
    return roll

In [15]:
def compute_roll_hash_arrays(server_seed_array,client_seed_array,nonce_array):
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(server_seed_array,
                                           client_seed_array,
                                           nonce_array)
    
    # Compute roll for hash using the hash which is the next element in array
    # Vectorize the function
    vectorized_calculate_roll_hash = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array_hash = vectorized_calculate_roll_hash(server_seed_array[1:],
                                           client_seed_array[:chain_length-1],
                                           nonce_array[:chain_length-1])
    return roll_array,roll_array_hash

    

In [16]:
# trans_36 = bytes((x ^ 0x36) for x in range(256))
# trans_36

In [17]:
# chain_length = 5110101 #1000001

# filename = "sha256_hashchain_b9556671f785fe935bee087665b4047e421ea4491a5e2021a8152cab0b74c953.npy"

# if KAGGLE:
#     filepath_client ="/kaggle/input/hash-file-generation-client-seed/"
# else:
#     filepath_client="data/"
    
# # filename_client=f'{filepath_client}sha256_hashchain_client_25M.npy'
# filename_client=f'{filepath_client}sha256_hashchain_client1.npy'


# # Define the number of CPU cores to use
# num_cores = 8

In [18]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [19]:
# server_seed_array=np.load(filename)
# print(len(server_seed_array))
# server_seed_array[0:10]

In [20]:
# client_seed_array=np.load(filename_client,
#                          allow_pickle=True,fix_imports=True,encoding='latin1')
# client_seed_array= client_seed_array[:chain_length]
# print(len(client_seed_array))
# client_seed_array[0:10]

In [21]:
# str_cat ='pppppp'
# # str_cat ='ppppppppppppppp'
# print(len(str_cat))
# client_seed_array_1=(str_cat + pd.Series(client_seed_array)).values
# client_seed_array_1[:2]

In [22]:
# # str_cat ='ppppppppppppppp'
# str_cat ='woxpwoxpwoxpwoxpwoxp'
# print(len(str_cat))
# client_seed_array_2=(str_cat + pd.Series(client_seed_array)).values
# client_seed_array_2[:2]

In [23]:
# np.random.seed(5000)
# low = 1500
# # nonce_array = np.random.randint(low, high=low+chain_length, 
# #                                 size=chain_length)

# nonce_array = np.arange(low,low+chain_length,1)
# np.random.shuffle(nonce_array)
# print(len(nonce_array))
# print(nonce_array[:10])
# pd.Series(nonce_array).nunique()

In [24]:
# roll_array,roll_array_hash=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array,
#                                        nonce_array)
# print(len(roll_array[0:10]))
# print(roll_array[0:100])
# print(len(roll_array_hash[0:10]))
# print(roll_array_hash[0:100])

In [25]:
# roll_array_1,roll_array_hash_1=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array_1,
#                                        nonce_array)
# print(len(roll_array_1[0:10]))
# print(roll_array_1[0:100])
# print(len(roll_array_hash_1[0:10]))
# print(roll_array_hash_1[0:100])

In [26]:
# roll_array_2,roll_array_hash_2=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array_2,
#                                        nonce_array)
# print(len(roll_array_2[0:10]))
# print(roll_array_2[0:100])
# print(len(roll_array_hash_2[0:10]))
# print(roll_array_hash_2[0:100])

In [27]:
# # Vectorize the function
# vectorized_calculate_roll = np.vectorize(calculate_roll)

# # Compute the roll values for the input arrays
# roll_array = vectorized_calculate_roll(server_seed_array,
#                                        client_seed_array,
#                                        nonce_array)

# # Print the output
# print(len(roll_array[0:10]))
# print(roll_array[0:100])

In [28]:
# # Vectorize the function
# vectorized_calculate_roll_hash = np.vectorize(calculate_roll)

# # Compute the roll values for the input arrays
# roll_array_hash = vectorized_calculate_roll_hash(server_seed_array[1:],
#                                        client_seed_array[:chain_length-1],
#                                        nonce_array[:chain_length-1])

# # Print the output
# print(len(roll_array_hash[0:10]))
# print(roll_array_hash[0:100])

In [29]:
ABOVE="Above"
BELOW="Below"
INTER ="Intermediate"

In [30]:
# Define a function to generate a single hash in the hash chain
def generate_hash(message):
    # Hash the message using SHA-256
    hash_obj = hashlib.sha256(message)

    # Get the hexadecimal representation of the hash
    hash_hex = hash_obj.hexdigest()

    # Return the hash
    return hash_hex

In [31]:
def generate_hash_chain(seed,chain_length):
    hash_list=[seed]
    message = seed.encode()
    for i in range(chain_length):
        hash_hex= generate_hash(message)
        # Write the hash to the CSV file
        hash_list.append(hash_hex)
        # Update the message with the current hash
        message = hash_hex.encode()
    return hash_list

def compute_multirolls(hash_list,client_seed,nonce):
    rolls=[]
    for cur_hash in hash_list:
        roll = calculate_roll(cur_hash,client_seed,nonce)
        rolls.append(roll)
    return rolls
def compute_multirolls_nonce(server_hash,client_seed,hash_list_nonce):
    rolls=[]
    for cur_hash_nonce in hash_list_nonce:
        roll = calculate_roll(server_hash,client_seed,cur_hash_nonce)
        rolls.append(roll)
    return rolls

In [32]:
chain_length = 1010101

In [33]:
%%time
message = b"3455dda4b3aecaa36d4687277766a079feebbb4ab01dc038bc8fb8a36ddad6aa"

client_seed_array=["3455dda4b3aecaa36d4687277766a079feebbb4ab01dc038bc8fb8a36ddad6aa"]
for i in range(chain_length):
    # Hash the current message using SHA-256
    hash_obj = hashlib.sha256(message)

    # Get the hexadecimal representation of the hash
    hash_hex = hash_obj.hexdigest()

    # Write the hash to the CSV file
    client_seed_array.append(hash_hex)

    # Update the message with the current hash
    message = hash_hex.encode()
    
client_seed_array = np.array(client_seed_array)   
client_seed_array[:5]

CPU times: user 964 ms, sys: 86.9 ms, total: 1.05 s
Wall time: 1.05 s


array(['3455dda4b3aecaa36d4687277766a079feebbb4ab01dc038bc8fb8a36ddad6aa',
       '57eea50e4484ba102d776506baf4b1dc0dcb3fc74fe2ab56b10b0f427cb6409e',
       '3e49b34f2d6c7116d92ac8818c94f87d5616d7f2aea5024c2f38413605dc321c',
       'a0731d74c9501894d16ba144bc10dd49e8439c7f88304bfdbe4a755870c46da3',
       '665122519e5835c8e7ebfa0ab5b1949d75ac2c9e3dc1a6965a4b6e26bfc4ed76'],
      dtype='<U64')

In [34]:
print(len(client_seed_array))

1010102


In [35]:
# str_cat ='ppppppppppppppp'
str_cat ='woxpwoxpwoxpwoxpwoxp'
print(len(str_cat))
client_seed_array_2=(str_cat + pd.Series(client_seed_array)).values
client_seed_array_2[:2]

20


array(['woxpwoxpwoxpwoxpwoxp3455dda4b3aecaa36d4687277766a079feebbb4ab01dc038bc8fb8a36ddad6aa',
       'woxpwoxpwoxpwoxpwoxp57eea50e4484ba102d776506baf4b1dc0dcb3fc74fe2ab56b10b0f427cb6409e'],
      dtype=object)

In [36]:
def predict_digit_pattern(server_hash,roll_hash,nonce,client_seed,
                        match_digit_arr,match_digit_indices,
                        mismatch_digit_arr=None,mismatch_digit_indices=None):

#     rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
    match = True
    
#     roll_first = rolls_list[0]
    factors = np.array([10000,1000,100,10])
    factors_filt = factors[match_digit_indices]
    for i,val in enumerate(factors_filt):
        match = match & (int((roll_hash % val) / (val / 10))==match_digit_arr[i])
        if not match: 
            break

    if mismatch_digit_arr is not None:
        factors_filt = factors[mismatch_digit_indices]
        for i,val in enumerate(factors_filt):
            match = match & (int((roll_hash % val) / (val / 10))!=mismatch_digit_arr[i])
            if not match: 
                break            

    return match
 

def predict_digit_output(server_hash,nonce,client_seed,
                         match_digit_arr,match_digit_count_arr,
                        hash_list_server,match_digit_indices=None):

    rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
    match = True
    
    #eliminate zero from all roll_hash
#     for roll in rolls_list:
#         str_roll = f'{roll:04d}'
#         cur_count = str_roll.count('0')
#         if cur_count!=0:
#             match=False
            
    for roll,match_digit,match_count,match_indices \
            in zip(rolls_list,match_digit_arr,match_digit_count_arr,match_digit_indices):
        str_roll = f'{roll:04d}'
        
        if match_digit_indices is None:
            str_roll_indexed = str_roll
        else:
            str_roll_indexed = ''
            for index in match_indices:
                str_roll_indexed = str_roll_indexed + str_roll[int(index)]
                
        cur_count = str_roll_indexed.count(match_digit)
        if cur_count!=match_count:
            match=False
            
    return match, rolls_list  

In [37]:
# test_file_random ='/kaggle/input/hash-generate-random-seeds/test_data_random.csv'
# test_seeds_2=pd.read_csv(test_file_random)
# print(test_seeds_2.shape)
# test_seeds_2.head()

In [38]:
# #create special client seed pattern array

# def get_hexval(val):
#     hexdata='abcdef'
#     if val >=10:
#         return hexdata[val-10]
#     else:
#         return str(val)

# str_spl_client = 'woxpwoxpwoxpwoxpwoxp066103c1b2a6ebe01cf30afd49a6b931278793fc457dee84510f03e11779d5be'
# spl_client_list=[]
# for index in range(20,20+64):
#     for val in range(16):
#         spl_client_list.append(str_spl_client[:index]+get_hexval(val)+str_spl_client[index+1:])
# spl_client_array = np.array(spl_client_list)
# print(len(spl_client_array))
# spl_client_array[1020:]

##### Machine Learning

Generate Data

In [39]:
def generate_data(test,client_seed_data,pass_state_reqd,
                    match_digit_arr,match_digit_count_arr,
                    is_match_digit_reqd=True,
                    mismatch_digit_arr=None,mismatch_digit_indices=None,
                    feature_chain_length=20,
                    is_data_hash=False,
                    match_digit_indices=None,
                    match_count_exp=1,
                    test_limit = None,
                    last_index=None,result_df=None,
                    print_client_scan=False,
                    hash_list_nonce=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'passed_state',
                                    'roll_actual','roll_hash_list','roll_hash_list_nonce',
                                         ])
    result_success=True
    dictionary_list=[]
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        
        if (server_count % 5)==0:
            print(f'server_count:{server_count}')
        
        if (test_limit is not None) and (server_count >= test_limit):
            break
        
        #for large volume of data, it is recommended to set i to entire lengthof client seeds
        #for every server seed
#         i = len(client_seed_data)-1 if last_index is None else last_index - 1
        
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
       
        if is_data_hash:
            cur_hash_list = list(test[server_count+1:server_count+1+feature_chain_length+1]['seed'])
        else:
            cur_hash_list = generate_hash_chain(cur_hash,feature_chain_length)
        
          
        fullclientscan=False
        match = False
        prev_match=""
        match_count=0
        
        client_start = i
        while match_count < match_count_exp and not fullclientscan:
            
            if print_client_scan  & (i % 1000==0) :
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i]
            client_index = i

            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
            
            if is_match_digit_reqd:
                match = predict_digit_pattern(cur_hash,roll_hash,cur_nonce,client_seed,
                            match_digit_arr,match_digit_indices,
                            mismatch_digit_arr=mismatch_digit_arr,
                            mismatch_digit_indices=mismatch_digit_indices)
        
            else:
                match = True
            # match, rolls_list = predict_digit_output(cur_hash,cur_nonce,client_seed,
            #              match_digit_arr,match_digit_count_arr,
            #             cur_hash_list,match_digit_indices=match_digit_indices)
            i-=1
            
            if i==client_start and not match:
                print()
                print(i,client_start)
                print(f'No further match for seed:{cur_seed} nonce:{cur_nonce} match count: {match_count}')
                print()
                fullclientscan=True
            if (i<0):
                i = len(client_seed_data)-1
       
            if match:
                
#                 print(f'Matched for seed:{cur_seed} i:{i} match count: {match_count}')
                rolls_list  = compute_multirolls(cur_hash_list,client_seed,cur_nonce)
                if hash_list_nonce is None:
                    hash_list_nonce = generate_hash_chain(str(cur_nonce),feature_chain_length)

                rolls_list_nonce  = compute_multirolls_nonce(cur_hash,client_seed,hash_list_nonce)

                match_count += 1
                
                result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                                | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
                if result_success:
                    last_success_index= i+1
#                 result = [result_success,cur_seed, cur_hash, cur_nonce,
#                                                client_seed,client_index,pass_state_reqd,
#                                                roll_seed_actual,rolls_list,rolls_list_nonce]
                

                dictionary_data = {'match': result_success,'seed':cur_seed, 
                                   'hash': cur_hash, 'nonce':cur_nonce,
                                    'client_seed':client_seed,'client_index':client_index,
                                      'passed_state':pass_state_reqd,
                                     'roll_actual':roll_seed_actual,
                                   'roll_hash_list':rolls_list,'roll_hash_list_nonce':rolls_list_nonce
                                  }
                dictionary_list.append(dictionary_data)
                
#                 result_df.loc[len(result_df.index)] = result
#                 print(f'server_count: {server_count} current result: {result_success, cur_nonce,pass_state_reqd,roll_seed_actual,roll_hash}')
    
    result_df = pd.DataFrame.from_dict(dictionary_list)

    return result_df


In [40]:
hashval = 'd807e64fa2f09e6403421135aa6d094dea9e34d6fe8d5b08e15e89b2e382528c'#'3e24f29a3ae70e54aedd86b0c68640050be7dace8ae679f1fb85d1325a56ddae'
actual_seed = '194d1847e72f934dac6a98526a2e41b27a4330e9d2ad729037d729467b3e9b21'
seed = hashval #'c77b3e783f094e255b0693f89094bed95ad9738d86f44fa4c09c8b0d58e2c73e'
nonce = 2826 #2390

# hashval = 'b022204a9aaaa67b75a924292d637bb07551f7517dca4b1b0a1d3965e7f971ac'#'3e24f29a3ae70e54aedd86b0c68640050be7dace8ae679f1fb85d1325a56ddae'
# actual_seed = '6a1f54767d9684db9851a372336906d00da9fe80a756b7718a76d959d896100c'
# seed = hashval #'c77b3e783f094e255b0693f89094bed95ad9738d86f44fa4c09c8b0d58e2c73e'
# nonce = 2831 #2390


feature_chain_length=60
train_hash_size = 2158#121
train_client_size = 2000 
test_client_size = 10000

In [41]:
cur_hash_list = generate_hash_chain(hashval,train_hash_size)

train_data = pd.DataFrame(columns=['seed','hash','nonce'])
start=24
train_data['seed']=np.array(cur_hash_list[start:-1])
train_data['hash']=np.array(cur_hash_list)[start+1:]
train_data['nonce'] = nonce

print(len(train_data))
train_data.head()

2134


,seed,hash,nonce
0,35696b22c790611bbd11cc0369c94bb0212f160956535fc3fd5e121a58916f3c,e0237d786d739f259565f7d9783b175cbd26e5003020fe740b935051b226a783,2826
1,e0237d786d739f259565f7d9783b175cbd26e5003020fe740b935051b226a783,6d6bb1fb8eb074a3f3dada3ab8046b3f82f3c1496561eb47b6b287725de7ba8b,2826
2,6d6bb1fb8eb074a3f3dada3ab8046b3f82f3c1496561eb47b6b287725de7ba8b,764dec4246ff6f89800022701291c49c010a3fe648bfaff828366f7715e6282e,2826
3,764dec4246ff6f89800022701291c49c010a3fe648bfaff828366f7715e6282e,bf701d178fae6ac64dfe1009df88397ba02a945375cddb0dbb9a5935a3cd078c,2826
4,bf701d178fae6ac64dfe1009df88397ba02a945375cddb0dbb9a5935a3cd078c,ac88304da822423921894e76cfed06e1c156fd56f99f78253f978a33f03bab8f,2826


In [42]:
PATTERN_99=False
HIGH_ANALYSIS=False

cur_digit = 9
cur_zero_digit = 0

In [43]:
OLD_VERSION_READ=False
READ_FROM_FILE=False

SAVE_FILE=False

In [44]:
%%time
# match_digit_arr = np.array(['999',])
# match_digit_count_arr=np.array([1])
# match_digit_indices = np.array(['012'])

if PATTERN_99:
    match_digit_arr=np.array([9,9])
    mismatch_digit_arr=np.array([9,9])
else:
    match_digit_arr=np.array([0,0])
    mismatch_digit_arr=np.array([0,0])
    
match_digit_indices = np.array([1,2])
match_digit_count_arr=np.array([1])
mismatch_digit_indices = np.array([0,3])

CPU times: user 10 µs, sys: 24 µs, total: 34 µs
Wall time: 39.3 µs


In [45]:
def save_cleaned_results_df(results_df,is_test,
                            file_suffix="",cleaned_suffix = True):
    nonce_suffix = "_nonce"
    rolls_df = results_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    rolls_df_nonce = results_df['roll_hash_list_nonce'].apply(pd.Series)
    rolls_df_nonce.columns = [f'roll_{i}{nonce_suffix}' for i in range(rolls_df.shape[1])]
    results_df_cleaned = pd.concat([results_df,rolls_df,rolls_df_nonce],axis=1)
    results_df_cleaned=results_df_cleaned.drop(['roll_hash_list','roll_hash_list_nonce'],axis=1)
    if cleaned_suffix:
        cleaned_suffix='cleaned_'
    else:
        cleaned_suffix=''
    if is_test:
        filename = f'data/{cleaned_suffix}results_df_test_{nonce}_{file_pattern_str}_pattern{file_suffix}.csv'
    else:
        filename = f'data/{cleaned_suffix}results_df_{nonce}_Large_{file_pattern_str}_pattern{file_suffix}.csv'
    if SAVE_FILE:        
        results_df_cleaned.to_csv(filename,index=False)
    return results_df_cleaned

In [46]:
%%time
if PATTERN_99:
    file_pattern_str = 'x99x'
else:
    file_pattern_str = 'x00x'
filename = f'data/results_df_{nonce}_Large_{file_pattern_str}_pattern.csv'

CPU times: user 2 µs, sys: 4 µs, total: 6 µs
Wall time: 10.5 µs


In [47]:
HIGH_TARGET= 4750 #8944
LOW_TARGET = 5250 #1056
ZERO_COL_CUTOFF = 4

In [48]:
def create_target_mask(data,bHighCheck=HIGH_ANALYSIS):
    if bHighCheck:
        mask = data['roll_actual'] > HIGH_TARGET
    else:
        mask = data['roll_actual'] < LOW_TARGET   
    return mask

Prediction

In [49]:
def gen_test_data(cur_nonce,cur_hash,cur_seed,file_suffix="",
                 is_match_digit_reqd =True,test_size=test_client_size):
    test_data = pd.DataFrame(columns=['seed','hash','nonce'])
    test_data.loc[0,'seed']=cur_seed
    test_data.loc[0,'hash']=cur_hash
    test_data.loc[0,'nonce'] = cur_nonce

    print(len(test_data))
    print(test_data.head())
    
    if PATTERN_99:
        match_digit_arr=np.array([9,9])
        mismatch_digit_arr=np.array([9,9])
    else:
        match_digit_arr=np.array([0,0])
        mismatch_digit_arr=np.array([0,0])

    match_digit_indices = np.array([1,2])
    match_digit_count_arr=np.array([1])
    mismatch_digit_indices = np.array([0,3]) 
    
    if PATTERN_99:
        file_pattern_str = 'x99x'
    else:
        file_pattern_str = 'x00x'
        
    filename = f'data/results_df_test_{nonce}_{file_pattern_str}_pattern{file_suffix}.csv'
    
    if  READ_FROM_FILE_TEST:
        results_df_test = pd.read_csv(filename)
#         results_df_test = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
#                                                         'roll_hash_list_nonce': pd.eval})
    else:

        cur_client_seed_array = client_seed_array_2[:1000001]
        results_df_test = None

        last_index_test =  len(cur_client_seed_array)-1 #1024
        start = 0

        results_df_test = generate_data(test_data,
                                    cur_client_seed_array,
                                     ABOVE,
                            match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
                            mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
                            is_match_digit_reqd=is_match_digit_reqd,
                            match_count_exp=test_size, 
                            feature_chain_length=feature_chain_length,
                            is_data_hash = False,
                            test_limit=1,
                            last_index=last_index_test,result_df=results_df_test,
                            print_client_scan=False)
        #Remove client seed duplicates from test data
        results_df_test= results_df_test.drop_duplicates(subset='client_seed')
        
    if not(READ_FROM_FILE_TEST) or OLD_VERSION_READ:
        results_df_test=save_cleaned_results_df(results_df_test,True,
                            file_suffix=file_suffix,cleaned_suffix= False)

#         results_df_test.to_csv(filename,index=False)
        
    print('Test Data Generation Completed')    
        
#         row = results_df_test.iloc[len(results_df_test)-1]
#         last_index_test = row['client_index']   

    return results_df_test
 

In [50]:
%%time
READ_FROM_FILE_TEST=False
# seed='dummy'
seed = generate_hash_chain(hashval,26)[25]
print('seed:',seed)
results_df_test=gen_test_data(nonce,hashval,seed,file_suffix="",
                              is_match_digit_reqd =False,
                              test_size=1000
                             )
print()
print(results_df_test[['seed','hash','nonce']].head(1))

seed: e0237d786d739f259565f7d9783b175cbd26e5003020fe740b935051b226a783
1
                                                               seed  \
0  e0237d786d739f259565f7d9783b175cbd26e5003020fe740b935051b226a783   

                                                               hash nonce  
0  d807e64fa2f09e6403421135aa6d094dea9e34d6fe8d5b08e15e89b2e382528c  2826  
server_count:0
Test Data Generation Completed

                                                               seed  \
0  e0237d786d739f259565f7d9783b175cbd26e5003020fe740b935051b226a783   

                                                               hash  nonce  
0  d807e64fa2f09e6403421135aa6d094dea9e34d6fe8d5b08e15e89b2e382528c   2826  
CPU times: user 1.05 s, sys: 1.26 ms, total: 1.05 s
Wall time: 1.05 s


In [51]:
# %%time
# READ_FROM_FILE_TEST=True
# #Generate hash test data
# print(f'{hashval=}')
# hash_l2=generate_hash(hashval.encode())
# print(f'{hash_l2=}')
# results_df_test_hash=gen_test_data(nonce,hash_l2,hashval,file_suffix="_hash")
# print()
# print(results_df_test_hash.head(1))

In [52]:
# %%time
# READ_FROM_FILE_TEST=False
# cur_seed = cur_hash_list[25]
# cur_hash = cur_hash_list[26]
# print(f'{cur_seed=}')
# print(f'{cur_hash=}')

# results_df_test_hash25=gen_test_data(nonce,cur_hash,cur_seed,file_suffix="_hash25")
# print()
# print(results_df_test_hash25.head(1))

In [53]:
# %%time
# results_df_test=save_cleaned_results_df(results_df_test,True)
# # results_df_test_hash=save_cleaned_results_df(results_df_test_hash,True)
# results_df_test_hash_cleaned=save_cleaned_results_df(results_df_test_hash,True,'_hash')
# # results_df_test_cleaned=save_cleaned_results_df(results_df_test_hash25,True)
# # results_df_test_cleaned=save_cleaned_results_df(results_df_test_hash24,True)

##### Feature Generation

In [54]:
def generate_roll_features_k(rolls_df,roll_start,roll_end,suffix="",stat_suffix=""):
    first_k_roll_cols = [f'roll_{i}{suffix}' for i in range(roll_start,roll_end+1)]
    rolls_df[f'count_gt_9000_k{stat_suffix}'] = (rolls_df[first_k_roll_cols] >= 9000).sum(axis=1)
    rolls_df[f'count_lt_1000_k{stat_suffix}'] = (rolls_df[first_k_roll_cols] < 1000).sum(axis=1)
    
    return rolls_df
    
def generate_roll_features(rolls_df,
                           roll_start=1, roll_end=20,k=4,
                          suffix="",stat_suffix=""):
    cols_roll = [f'roll_{i}{suffix}' for i in range(roll_start,roll_end+1)]
#     print(cols_roll)
    
    if (roll_start==1) & (roll_end==20):
        stat_suffix = suffix
    else:
        stat_suffix = f'_{roll_start}_{roll_end}{suffix}'
    
    rolls_df[f'count_lt_1000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==0).sum(axis=1)
    rolls_df[f'count_gt_9000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
    rolls_df[f'count_gt_8000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
    rolls_df[f'count_gt_7000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
    rolls_df[f'count_gt_6000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
    rolls_df[f'count_gt_5000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
    rolls_df[f'count_gt_4000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
    rolls_df[f'count_gt_3000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
    rolls_df[f'count_gt_2000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
    rolls_df[f'count_gt_1000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    
    rolls_df[f'count_hi{stat_suffix}'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
    rolls_df[f'count_lo{stat_suffix}'] = (rolls_df[cols_roll] < 4750).sum(axis=1)    
    rolls_df[f'count_lo_hi{stat_suffix}'] = rolls_df[f'count_hi{stat_suffix}'] \
                                            + rolls_df[f'count_lo{stat_suffix}']   


    rolls_df=generate_roll_features_k(rolls_df,roll_start,roll_start+k,
                                      suffix=suffix,stat_suffix=stat_suffix)
    
    roll_list_strs = np.full(len(rolls_df),"")
    for col in cols_roll:
        roll_list_strs = rolls_df[col].astype('str') + roll_list_strs
    
    rolls_df[f'total_dig_9{stat_suffix}']= roll_list_strs.str.count('9')
    rolls_df[f'total_dig_0{stat_suffix}']= roll_list_strs.str.count('0')
    
    rolls_df[f'roll_mean{stat_suffix}'] = (rolls_df[cols_roll]).mean(axis=1)
    rolls_df[f'roll_std{stat_suffix}'] = (rolls_df[cols_roll]).std(axis=1)
     
    print(f'Roll Features from {roll_start} to {roll_end} {suffix} completed' )
    
    return rolls_df

#Modification: 
#1. supported both raw roll_hash_list and already generated roll columns if present
#2. added client index
def generate_features_full(initial_df,istrain,feature_chain_length):
    
    nonce_suffix = '_nonce'
    if 'roll_1' in list(initial_df.columns):
        print('roll columns present')
        rolls_cols = [f'roll_{i}' for i in range(feature_chain_length+1)] 
        rolls_df = initial_df[rolls_cols]
        rolls_cols_nonce = [f'roll_{i}{nonce_suffix}' for i in range(feature_chain_length+1)] 
        rolls_df_nonce = initial_df[rolls_cols_nonce]
    else:
        print('roll columns not present')
        rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
        rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]    
        rolls_df_nonce = initial_df['roll_hash_list_nonce'].apply(pd.Series)
        rolls_df_nonce.columns = [f'roll_{i}{nonce_suffix}' for i in range(rolls_df_nonce.shape[1])] 
    
    #generate top 20 hash features
    rolls_df=generate_roll_features(rolls_df,roll_start=1,
                                    roll_end=20, k=4)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix,roll_start=1,
                                    roll_end=20 , k=4)
    
    #generate 25's hash features
    roll_start =25
    roll_end = 50
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)  

    #generate 50's hash features
    roll_start =50
    roll_end = 60
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,roll_start=roll_start,
                                    roll_end=roll_end, k=5)      
    #generate for full hash features
    roll_start =1
    roll_end = feature_chain_length
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,roll_start=roll_start,
                                    roll_end=roll_end, k=5)        
    if istrain:
        features_df = pd.concat([initial_df[['roll_actual','seed',
                                             'hash','client_seed','client_index']],
                                 rolls_df,
                                 rolls_df_nonce],axis=1)
        mask = create_target_mask(features_df)
            
        features_df['target'] =0
        features_df.loc[mask,'target'] = 1
        features_df['roll_actual']=features_df['roll_actual'].astype('int')
        print(features_df['target'].value_counts())
    else:
        features_df = pd.concat([initial_df[['roll_actual','seed',
                                             'hash','client_seed','client_index']],
                                             rolls_df,rolls_df_nonce],axis=1)
    return features_df


In [55]:
%%time
test = generate_features_full(results_df_test,False,feature_chain_length)
print(len(test))
test.head()

roll columns present
Roll Features from 1 to 20  completed
Roll Features from 1 to 20 _nonce completed
Roll Features from 25 to 50  completed
Roll Features from 25 to 50 _nonce completed
Roll Features from 50 to 60  completed
Roll Features from 50 to 60 _nonce completed
Roll Features from 1 to 60  completed
Roll Features from 1 to 60 _nonce completed
1000
CPU times: user 316 ms, sys: 0 ns, total: 316 ms
Wall time: 315 ms


,roll_actual,seed,hash,client_seed,client_index,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_non

In [56]:
# %%time
# test_hash = generate_features_full(results_df_test_hash,False,feature_chain_length)
# print(len(test))
# test_hash.head()

#### Machine Learning

In [57]:
USE_STAT_FEATS = True

In [58]:

if USE_STAT_FEATS:
    exclude_stat_cols=[]
else:
#     exclude_stat_cols = ['roll_mean','roll_std','roll_mean_nonce','roll_std_nonce']
    exclude_stat_col_starts = ['roll_mean','roll_std','roll_mean_nonce','roll_std_nonce']
    exclude_stat_cols=[]
    for stat_col in exclude_stat_col_starts:
        exclude_stat_cols += [col for col in train.columns if col.startswith(stat_col)]
    print(exclude_stat_cols)
exclude_cols = []#[f'roll_{i}' for i in range(11,feature_chain_length)]
exclude_cols_nonce =[]# [f'roll_{i}_nonce' for i in range(11,feature_chain_length+1)]
exclude_count_cols = [f'count_gt_{i}000' for i in [1,2,3,4,6]]
exclude_count_cols_nonce = [f'count_gt_{i}000_nonce' for i in [1,2,3,4,6]]
othercols = ['roll_actual','client_seed','seed','hash','client_index'] + exclude_stat_cols

all_exclude_cols = othercols+exclude_cols+exclude_cols_nonce+\
                    exclude_count_cols+exclude_count_cols_nonce
                  
features = [col for col in test.columns if col not in all_exclude_cols]
print(len(features))
print(features)

264
['roll_0', 'roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50', 'roll_51', 'roll_52', 'roll_53', 'roll_54', 'roll_55', 'roll_56', 'roll_57', 'roll_58', 'roll_59', 'roll_60', 'count_lt_1000', 'count_gt_9000', 'count_gt_8000', 'count_gt_7000', 'count_gt_5000', 'count_hi', 'count_lo', 'count_lo_hi', 'count_gt_9000_k', 'count_lt_1000_k', 'total_dig_9', 'total_dig_0', 'roll_mean', 'roll_std', 'count_lt_1000_25_50', 'count_gt_9000_25_50', 'count_gt_8000_25_50', 'count_gt_7000_25_50', 'count_gt_6000_25_50', 'c

In [59]:
import xgboost as xgb
# import matplotlib.pyplot as plt # for plotting graphs
# import seaborn as sns # for plotting graphs
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, f1_score,precision_recall_curve

Train Validation Split

In [60]:
#This function is useful to convert the predicted probabilities into labels 
#so that F1 score is optimized
#It first determines the probability threshold using precision recall curve at which F1 score is optimized and 
#then generate the optimized labels based on the determined probability threshold
def get_opt_cutoff_prec(labels,preds):
    precision, recall, thresholds  = precision_recall_curve(labels, preds)
    f1_score= 2*((precision*recall)/(precision+recall))
    
    f1_score = f1_score[:len(thresholds)]
    f1_score[np.isnan(f1_score)]=0
#     optimal_idx = np.nanargmax(precision[:len(precision)-1])
#     print(f'precision borders:{precision[0],precision[len(precision)-2]}')
#     print(f'recall borders:{recall[0],recall[len(recall)-2]}')
#     print(f'f1_score size:{len(f1_score)} recall size:{len(recall)}')
#     print(f'{precision=}')
#     print(f'{recall=}')
#     print(f'{f1_score=}')
#     print(f'{thresholds=}')
#     print(f'optimal_idx:{optimal_idx} precision size:{len(precision)} thresholds size:{len(thresholds)}')
    optimal_idx = np.nanargmax(f1_score)
    optimal_threshold = thresholds[optimal_idx]
#     return optimal_threshold, precision[optimal_idx]
    return optimal_threshold, f1_score[optimal_idx]

def convert_probtolabels(preds,cutoff=0.5):
    y_bin= preds.copy()
    y_bin[preds>cutoff] = 1
    y_bin[preds<=cutoff] = 0
    y_bin=y_bin.astype(int)

    return y_bin

In [61]:
def generate_probs(multi_models,model,data,features):
    if multi_models:
        probs= np.zeros(len(data))
        for i,model_ind in enumerate(model):
            cur_prob= model_ind.predict_proba(data[features])[:,1] 
            probs+= cur_prob / len(model)
    else: 
        probs= model.predict_proba(test_filt_df[features_test])[:,1]
    data['probs']=probs
    return data

##### Prediction of Test 

In [62]:
CONT_ANALYSIS=False
IS_PROD=False

In [63]:
# actual_seed = cur_hash_list[25]
print(actual_seed)
# Vectorize the function
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(actual_seed,
                                       results_df_test['client_seed'],
                                       nonce)

results_df_test['roll_actual']=roll_array
#     test_manual = generate_test_features(results_df_test)
test = generate_features_full(results_df_test,False,feature_chain_length)

194d1847e72f934dac6a98526a2e41b27a4330e9d2ad729037d729467b3e9b21
roll columns present
Roll Features from 1 to 20  completed
Roll Features from 1 to 20 _nonce completed
Roll Features from 25 to 50  completed
Roll Features from 25 to 50 _nonce completed
Roll Features from 50 to 60  completed
Roll Features from 50 to 60 _nonce completed
Roll Features from 1 to 60  completed
Roll Features from 1 to 60 _nonce completed


In [64]:
# mean_imp = np.zeros(len(features)-1)
# imp_df = pd.DataFrame()
# imp_df['feature'] = [col for col in features if col!='target']
# for i,model in enumerate(xgb_models):
#     cur_imp = model.feature_importances_
#     imp_df[f'importance_m{i}'] = cur_imp
# #     print(cur_imp)
#     mean_imp += cur_imp / len(xgb_models)

# # print(mean_imp)
# imp_df['importance'] = mean_imp
# imp_df = imp_df.sort_values('importance',ascending=False).reset_index(drop=True)
# imp_df

In [65]:
# %%time
# test_hash25 = generate_features_full(results_df_test_hash25,False,feature_chain_length)
# test_hash25.head()

In [66]:
# %%time
# test_hash24 = generate_features_full(results_df_test_hash24,False,feature_chain_length)
# test_hash24.head()

In [67]:
# hash_imp_name='roll_54'
# nonce_suffix = '_nonce'
# if hash_imp_name.find(nonce_suffix)!=-1:
#     is_nonce=True
#     hash_imp_name=hash_imp_name.replace(nonce_suffix,'')

# hash_no = int(hash_imp_name.replace('roll_',''))
# hash_no

In [68]:
# hash_imp_count=5
# roll_imp_feats = [col for col in imp_df['feature'].values if (col.startswith('roll_'))
#                                 and ('std' not in col ) and ('mean' not in col )][:hash_imp_count] 
# roll_imp_feats


In [69]:
%%time
READ_FROM_FILE_TEST=False

hash_list_nonce = generate_hash_chain(str(nonce),5025)
# cur_hash_list = generate_hash_chain(hashval,55025)
print('Hash List completed')

# hash_imp_count=10
hash_imp_names= ['roll_1000_nonce','roll_2000_nonce','roll_3000_nonce','roll_4000_nonce','roll_5000_nonce']
hash_imp_count = len(hash_imp_names)
# hash_imp_count=5
# # test_hash_imp = np.zeros((hash_imp_count))
# hash_imp_names = [col for col in imp_df['feature'].values if (col.startswith('roll_'))
#                                 and ('std' not in col ) and ('mean' not in col )][:hash_imp_count] 

print(f'Top Hash Important Features: {hash_imp_names}')
nonce_suffix = '_nonce'
test_hash_imp = []
for i in range(hash_imp_count):

    hash_imp_name = hash_imp_names[i]
    
    print(f'\n********* Processing Test Hash {hash_imp_name} ******************')

    if hash_imp_name.find(nonce_suffix)!=-1:
        is_nonce=True
        hash_imp_name=hash_imp_name.replace(nonce_suffix,'')
        nonce_no = int(hash_imp_name.replace('roll_',''))
        cur_nonce = hash_list_nonce[nonce_no]
        hash_no = 0
        file_suffix =f"_nonce{nonce_no}"
        print(f'{cur_nonce=}')
    else:
        hash_no = int(hash_imp_name.replace('roll_',''))
        cur_nonce= nonce
        file_suffix =f"_hash{hash_no}"

    cur_seed = cur_hash_list[hash_no]
    cur_hash = cur_hash_list[hash_no+1]
    print(f'{file_suffix=}')
    print(f'{hash_no=}')
    print(f'{cur_seed=}')
    print(f'{cur_hash=}')

    results_df_test_hash_imp=gen_test_data(cur_nonce,cur_hash,cur_seed,
                                           file_suffix=file_suffix,
                                           is_match_digit_reqd =False,test_size=1000,
#                                         file_suffix='NO_PATTERN_'+file_suffix
                                          )
#     results_df_test_hash_imp=save_cleaned_results_df(results_df_test_hash_imp,True,
#                                                 file_suffix=file_suffix)
    print()
#     print(results_df_test_hash_imp.head(1))
    cur_test_hash_imp = generate_features_full(results_df_test_hash_imp,False,feature_chain_length)
    test_hash_imp.append(cur_test_hash_imp)

Hash List completed
Top Hash Important Features: ['roll_1000_nonce', 'roll_2000_nonce', 'roll_3000_nonce', 'roll_4000_nonce', 'roll_5000_nonce']

********* Processing Test Hash roll_1000_nonce ******************
cur_nonce='986e75317ed3fb4e62ddb34704804fc5af736fb73de9f72dfd702e4220eb3d1c'
file_suffix='_nonce1000'
hash_no=0
cur_seed='d807e64fa2f09e6403421135aa6d094dea9e34d6fe8d5b08e15e89b2e382528c'
cur_hash='63de90c2cc29b88c67d759e125d4d60fba37d83863fb15df5f2f1710b23ccd2e'
1
                                                               seed  \
0  d807e64fa2f09e6403421135aa6d094dea9e34d6fe8d5b08e15e89b2e382528c   

                                                               hash  \
0  63de90c2cc29b88c67d759e125d4d60fba37d83863fb15df5f2f1710b23ccd2e   

                                                              nonce  
0  986e75317ed3fb4e62ddb34704804fc5af736fb73de9f72dfd702e4220eb3d1c  
server_count:0
Test Data Generation Completed

roll columns present
Roll Features from 1 to 20  

In [70]:
# test_hash_imp[0].head(25)

In [71]:
idx=4
mask = create_target_mask(test_hash_imp[idx])
print(len(test_hash_imp[idx]))
print(len(test_hash_imp[idx][mask]))

1000
520


In [72]:
# hash_imp_count = len(test_hash_imp)
# hash_imp_names += hash_imp_names_2
# print(hash_imp_names)
# hash_imp_count

In [73]:
# hash_imp_names=['roll_33_nonce', 'roll_3_nonce','roll_49','roll_42_nonce','roll_57_nonce',
#  'roll_55', 'roll_48_nonce', 'roll_59_nonce', 'roll_44_nonce',
#  'roll_53_nonce', 'roll_51', 'roll_6_nonce', 'roll_5_nonce', 'roll_21_nonce', 'roll_56_nonce',
#         'roll_34_nonce', 'roll_41', 'roll_25_nonce', 'roll_0_nonce', 'roll_45_nonce']
# hash_imp_count = len(test_hash_imp)


In [74]:
# test_hash1 = test_hash.copy()
# test_hash = test_hash24.copy()
# test_hash = test_hash1.copy()

In [75]:
test_filt=test.copy()
# test_filt_hash=test_hash.copy()
features_test = [col for col in features if col not in ['target','probs']]

In [76]:
def gen_hash_cutoff_df(models,feature_test,test_hash,k_bin_size=600,quant=0.95):
    test_hash_probs = generate_probs(True,models,test_hash,features_test)
    cutoffs = []
    ratios =[]
    totals =[]
    print(test_hash_probs['probs'].describe())
    cutoffs = np.linspace(test_hash_probs['probs'].min(),
                          test_hash_probs['probs'].quantile(quant),k_bin_size)
    for i,cutoff in enumerate(cutoffs[:len(cutoffs)-1]):

        mask = (test_hash_probs['probs']>=cutoff) & (test_hash_probs['probs']<=cutoffs[i+1])
        mask2=create_target_mask(test_hash_probs)

        total = len(test_hash_probs[mask])
        if total==0:
            ratio=0
            success=0
        else:
            success = len(test_hash_probs[mask & mask2])
            ratio = success / total
        ratios.append(ratio)
        totals.append(total)
    #     print(ratio,success,total)

    print(len(ratios),len(cutoffs))
    df=pd.DataFrame()
    df['cutoff']=cutoffs[:len(cutoffs)-1]
    df['cutoff_2']=cutoffs[1:len(cutoffs)]
    df['ratio'] =ratios
    df['total'] =totals

    df = df.sort_values(['ratio','cutoff'],ascending=[False,False]).reset_index(drop=True)
#     print(df[:5].mean())
#     print(df[:10].mean())
    return df

In [77]:
# test_hash_probs = generate_probs(True,xgb_models,test_hash,features_test)
# cutoffs = []
# ratios =[]
# totals =[]
# print(test_hash_probs['probs'].describe())
# cutoffs = np.linspace(test_hash_probs['probs'].min(),
#                       test_hash_probs['probs'].quantile(0.95),600)
# for i,cutoff in enumerate(cutoffs[:len(cutoffs)-1]):
# #     cutoff=0.4
# #     diff = 0.01
# #     print(cutoff,cutoffs[i+1])
#     mask = (test_hash_probs['probs']>=cutoff) & (test_hash_probs['probs']<=cutoffs[i+1])
#     mask2=create_target_mask(test_hash_probs)
    
# #     mask2 = test_hash_probs['roll_actual']>=9000
#     total = len(test_hash_probs[mask])
#     if total==0:
#         ratio=0
#         success=0
#     else:
#         success = len(test_hash_probs[mask & mask2])
#         ratio = success / total
#     ratios.append(ratio)
#     totals.append(total)
# #     print(ratio,success,total)

# print(len(ratios),len(cutoffs))
# df=pd.DataFrame()
# df['cutoff']=cutoffs[:len(cutoffs)-1]
# df['cutoff_2']=cutoffs[1:len(cutoffs)]
# df['ratio'] =ratios
# df['total'] =totals

# df = df.sort_values('ratio',ascending=False).reset_index(drop=True)
# print(df[:5].mean())
# print(df[:10].mean())
# df.head(10)

In [78]:
# df

In [79]:
# df.describe()

In [80]:
# print(len(df[df['total']==1]))
# print(len(df[(df['total']==1) & (df['ratio']>0)]))
# df[df['total']==1]

In [81]:
# df[df['ratio']<0.12].index[0]

In [82]:
# test['roll_actual']=test[f'roll_actual_orig'].copy()

In [83]:
def gen_all_k(models,features_test,data,test_hash_cutoff_df,target_total=1,is_k_data_req=True):
    test_probs = generate_probs(True,models,data,features_test)
    top_k=[]
    all_k=[]
    test_filt_all=None
    for k in range(len(test_hash_cutoff_df)):
        row = test_hash_cutoff_df.iloc[k]
    #     print(row['ratio'],row['cutoff'],row['cutoff_2'])
        mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
        mask2 = create_target_mask(test_probs)    
        test_filt = test_probs[mask]
        

        total = len(test_filt)
        if total==0:
            success=0
            ratio=0
        else:
            success = len(test_probs[mask & mask2])
            ratio = success / total
        if total==target_total:
            all_k.append(k) 
            if is_k_data_req:
#                 cur_test = test_filt[['roll_actual','client_seed','probs']]
                cur_test = test_filt
                cur_test['k']=k
                if test_filt_all is None:
                    test_filt_all = cur_test
                else:
                    test_filt_all = pd.concat([test_filt_all,cur_test],axis=0)
        if ratio>=0.12:
            if total==target_total:
                top_k.append((k,ratio,total))
    #     print(ratio,success,total,row['cutoff'],k)
    print(f'{len(all_k)=}')
#     print(all_k)
    print(f'{len(top_k)=}')
    print(top_k)
#     mask = (test_hash_probs['probs']>=row['cutoff']) & (test_hash_probs['probs']<=row['cutoff_2'])
#     test_filt_hash  = test_hash_probs[mask]   
    
    return all_k,top_k,test_filt_all

###### Normal Test Hash based Match Pos df generation

In [84]:
test.head()

,roll_actual,seed,hash,client_seed,client_index,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_non

In [85]:
# test_copy = test.copy()

In [86]:
# test_no_pattern = test.copy()

In [87]:
# test = test_copy.copy()
# test = test_no_pattern.copy()

In [88]:
# hash_count = 24
# # test = train[train['seed']==cur_hash_list[hash_count]].copy()
# test = train[train['seed'].isin(cur_hash_list[hash_count:hash_count+1])].copy()
# test = test.drop('target',axis=1)
# print(len(test))

In [89]:
# test = test_hash_imp_val[0].copy()
# print(len(test))

In [90]:
# test_hash_imp = test_hash_imp[10:20]+test_hash_imp[0:9]

In [91]:
len(test_hash_imp)

5

In [92]:
test_all_k = test

In [93]:
# test_all_k = generate_probs(True,xgb_models,test,features_test).copy()
# # test_all_k['k']=test_all_k['probs'].rank(method='min',ascending=False)
# print(len(test_all_k))

In [94]:
%%time

# k_bin_size=600

test_all_k_imp =[-1]*hash_imp_count


for hash_idx in range(hash_imp_count):

    print(f'***************** Test Hash {hash_imp_names[hash_idx]} ***************** ')

    
#     test_all_k_imp[hash_idx] = generate_probs(True,xgb_models,test_hash_imp[hash_idx],
#                                               features_test).copy()
    test_all_k_imp[hash_idx] =test_hash_imp[hash_idx]
#     test_all_k_imp[hash_idx]['k']=test_all_k_imp[hash_idx]['probs'].rank(method='min',ascending=False)


***************** Test Hash roll_1000_nonce ***************** 
***************** Test Hash roll_2000_nonce ***************** 
***************** Test Hash roll_3000_nonce ***************** 
***************** Test Hash roll_4000_nonce ***************** 
***************** Test Hash roll_5000_nonce ***************** 
CPU times: user 33 µs, sys: 21 µs, total: 54 µs
Wall time: 54.6 µs


In [95]:
# %%time

# k_bin_size=10000

# print(f'***************** Bin {k_bin_size} ***************** ')
# df= gen_hash_cutoff_df(xgb_models,features_test,test_hash,
#                             k_bin_size=k_bin_size,quant=0.95)
# print('Test Hash Cutoff Df Stats')
# print('Total size:',len(df[df['total']==1]))
# print('Success size:',len(df[(df['total']==1) & (df['ratio']>0)]))

In [96]:
# all_k,top_k,test_all_k = gen_all_k(xgb_models,features_test,
#                                 test,df,target_total=1)

# top_k_ele = [x[0] for x in top_k]
# print(top_k_ele)
# print('Test Size:',len(test_all_k))
# test_all_k.head()

In [97]:
# df[df['total']==1]

In [98]:
# key_cols = ['k','roll_actual','probs','roll_0','roll_1','client_seed']
# other_cols = [col for col in test_all_k if col not in key_cols]
# test_all_k[key_cols+other_cols]

In [99]:
# key_cols = ['k','roll_actual','client_seed','probs','roll_0','roll_1']
# other_cols = [col for col in test_all_k if col not in key_cols]
# test_all_k[test_all_k['k'].isin(top_k_ele)][key_cols+other_cols]

In [100]:
# # test_sel=test_all_k[:5]
# test_sel=test_all_k.sort_values('probs',ascending=False)[:5]
# test_sel

In [101]:
# top_k_ele = [x[0] for x in top_k]
# print(top_k_ele)
# matches = set(list(test_sel.k)).intersection(set(top_k_ele))
# print(f'{len(matches)}')
# print(matches)

Important Feats Top K Elements

In [102]:
# %%time

# # k_bin_size=600

# all_k_imp =[-1]*hash_imp_count
# top_k_imp =[-1]*hash_imp_count
# test_all_k_imp =[-1]*hash_imp_count
# top_k_ele_imp =[-1]*hash_imp_count

# df= gen_hash_cutoff_df(xgb_models,features_test,test_hash,
#                             k_bin_size=k_bin_size,quant=0.95)
# print('Test Hash Cutoff Df Stats')
# print('Total size:',len(df[df['total']==1]))
# print('Success size:',len(df[(df['total']==1) & (df['ratio']>0)]))

# for hash_idx in range(hash_imp_count):

#     print(f'***************** Test Hash {hash_imp_names[hash_idx]} ***************** ')

#     all_k_imp[hash_idx],top_k_imp[hash_idx],test_all_k_imp[hash_idx] = gen_all_k(xgb_models,features_test,
#                                                                         test_hash_imp[hash_idx],
#                                                                         df,target_total=1)

#     top_k_ele_imp[hash_idx] = [x[0] for x in top_k_imp[hash_idx]]


In [103]:
# top_k_ele_imp

In [104]:
# test_all_k_imp[0].head()

END

In [105]:
def get_sim_summary(test_filt_all_k,
                       is_imp_feats,imp_feats_count,
                       is_rank_pct=False):

    exclude_cols=['roll_actual',
        'roll_mean_rank','roll_mean_summary_init','roll_mean_rank_init',
                'roll_mean_summary','ratio_group_roll_rank','ratio_group_roll_rank_desc']
    if is_imp_feats:
#         print('imp_feats_count:',imp_feats_count)
#         hash_groups =[ [col for col in imp_df['feature'].values if (col.startswith('roll_')) \
#                        & (col not in exclude_cols)][:imp_feats_count] ]
        hash_groups =[ [col for col in test_filt_all_k.columns if (col.startswith('roll_')) \
                       & (col not in exclude_cols)] ]    
    else:
        hash_groups = [list(range(1,11)),list(range(25,36)),list(range(50,60)),list(range(100,111)) ]
    hash_groups_flat = [item for sublist in hash_groups for item in sublist]
#     print(hash_groups)
    
    
    if HIGH_ANALYSIS:
        success = (test_filt_all_k[hash_groups_flat] > HIGH_TARGET).sum(axis=1)
    else:
        success = (test_filt_all_k[hash_groups_flat] < LOW_TARGET).sum(axis=1)
    
    ratio_mean_init = success / len(hash_groups_flat)
    
    roll_mean_init = test_filt_all_k[hash_groups_flat].mean(axis=1)
    test_filt_all_k['ratio_mean_summary_init']=ratio_mean_init
    test_filt_all_k['roll_mean_summary_init']=roll_mean_init
    test_filt_all_k['ratio_mean_rank_init']= test_filt_all_k['ratio_mean_summary_init'].rank(method='min',
                                                                                           pct=is_rank_pct,
                                                                                           ascending=True)
    test_filt_all_k['roll_mean_rank_init']= test_filt_all_k['roll_mean_summary_init'].rank(method='min',
                                                                                           pct=is_rank_pct,
                                                                                           ascending=True)
    
    rev_cols = [col for col in hash_groups_flat if col not in ['roll_mean']] + \
                ['roll_mean_summary_init']
#     print('rev cols:',rev_cols)
    roll_mean = test_filt_all_k[rev_cols].mean(axis=1)
    test_filt_all_k['roll_mean']=roll_mean
    
    if HIGH_ANALYSIS:
        success = (test_filt_all_k[rev_cols] > HIGH_TARGET).sum(axis=1)
    else:
        success = (test_filt_all_k[rev_cols] < LOW_TARGET).sum(axis=1)
    
    ratio_mean = success / len(rev_cols)
    test_filt_all_k['ratio_mean_summary']=ratio_mean

    
#     test_filt_all_k['k']=test_filt_all_k['k']
    
    test_filt_all_k['ratio_group_roll_rank_init']= test_filt_all_k.groupby('ratio_mean_summary_init') \
                                                            ['roll_mean_summary_init'] \
                                                            .rank(method='min', \
                                                             pct=is_rank_pct)   
    test_filt_all_k['ratio_group_roll_rank_desc_init']= test_filt_all_k.groupby('ratio_mean_summary_init') \
                                                                        ['roll_mean_summary_init'] \
                                                                        .rank(method='min', \
                                                                              ascending=False, \
                                                                              pct=is_rank_pct)   
    
    test_filt_all_k['ratio_mean_rank']= test_filt_all_k['ratio_mean_summary'].rank(method='min',
                                                                           ascending=True,
                                                                            pct=is_rank_pct)
    test_filt_all_k['roll_mean_rank']= test_filt_all_k['roll_mean'].rank(method='min',
                                                                         pct=is_rank_pct,
                                                                           ascending=True)
    test_filt_all_k['ratio_group_roll_rank']= test_filt_all_k.groupby('ratio_mean_summary') \
                                                                    ['roll_mean'] \
                                                                    .rank(method='min', \
                                                                    pct=is_rank_pct)   
    test_filt_all_k['ratio_group_roll_rank_desc']= test_filt_all_k.groupby('ratio_mean_summary') \
                                                                        ['roll_mean'] \
                                                                        .rank(method='min', \
                                                                        ascending=False, \
                                                                        pct=is_rank_pct)   
#     test_filt_all_k['probs_rank']= test_filt_all_k['probs'].rank(method='min',pct=is_rank_pct,
#                                                                 ascending=False)
#     test_filt_all_k['ratio_group_probs_rank']= test_filt_all_k.groupby('ratio_mean_summary')['probs'] \
#                                                                             .rank(method='min', \
#                                                                                   pct=is_rank_pct, \
#                                                                                   ascending=False)    
#     for col in hash_groups_flat:
#         test_filt_all_k[f'{col}_rank']=test_filt_all_k[col].rank(method='min',
#                                                                 ascending=True,
#                                                                 pct=is_rank_pct)
    #     test_filt_all_k=sim_summary.reset_index(drop=True)
    
    return test_filt_all_k
    

In [106]:
# test_orig= test.copy()
# test_hash_orig = test_hash.copy()

In [107]:
# count_all_k = len(all_k)
# if count_all_k<10:
#     print(f'PROJECT WARNING: number of target records is {count_all_k} which is less than 10 ')
# count_top_k = len(top_k)    
# if count_top_k<2:
#     print(f'PROJECT WARNING: number of probable success is {count_top_k} which is less than 2')  
# hash_success = len(df[(df['total']==2) & (df['ratio']>0)])    
# if hash_success<2:
#     print(f'PROJECT WARNING: number of hash success is {hash_success} which is less than 2')

In [108]:
# print(len(test))
# mask = test.duplicated(subset='client_seed')
# print(len(test[mask]))
# print(test[mask]['client_seed'].nunique())

# print(len(test_hash))
# mask = test_hash.duplicated(subset='client_seed')
# print(len(test_hash[mask]))
# print(test_hash[mask]['client_seed'].nunique())

In [109]:
# print(all_k)
# print(top_k_ele)

In [110]:
def get_nlargest_ind(arr,n=10,asc=False):
    if not asc:
        ind = np.argpartition(arr, -n)[-n:]
    else:
        ind = np.argpartition(arr, -n)[:n]
    
    #     print(ind)

    topn= arr[ind]
#     print(topn)

    sorted_val=  np.argsort(arr[ind])
    if not asc:
        sorted_val=sorted_val[::-1]
#     print(sorted_val)
    sorted_ind  = ind[sorted_val]
    
    return sorted_ind



def get_top_success_count(probs,y,top_n=10,asc=False):
    
    top_prob_ind = get_nlargest_ind(probs,n=top_n,asc=asc)
    top_y = y[top_prob_ind]
    
#     print(f'{top_prob_ind=}')
#     print(f'{top_y=}')
#     print(f'{probs[top_prob_ind]}')
    
    total = top_n
    match_pos_arr = np.nonzero(top_y==1)[0]+1
    success = len(top_y[top_y==1])
    ratio = success / total
    
#     print(f'total:{total} success:{success} ratio:{ratio}')
    
    return success, ratio,top_prob_ind,match_pos_arr

def get_top_auc_score(probs,y,top_n=10,asc=False):
    
    top_prob_ind = get_nlargest_ind(probs,n=top_n,asc=asc)
    top_y = y[top_prob_ind]
    top_probs = probs[top_prob_ind]
    
    score = roc_auc_score(top_y, top_probs)
    
    return score

def cust_top_n_matches(y_pred, dtrain):
    y_true = dtrain.get_label()
    top_preds = 100
    success, ratio,top_prob_ind,match_pos_arr=get_top_success_count(y_pred,
                                                                    y_true,
                                                                  top_n=top_preds,asc=False)
    match_count = len(match_pos_arr)
    error = top_preds - match_count

    return 'top_n_matches', error

def cust_top_n_auc(y_pred, dtrain):
    y_true = dtrain.get_label()
    top_preds = 100
    auc_score=get_top_auc_score(y_pred,y_true,top_n=top_preds,asc=False)
    error = 1 - auc_score
    return 'top_n_matches', error

def gen_top_mean_data_ml(test_combined_imp,features_test,
                        n_estimators =100,asc=False,top_n=10,
                         booster = 'gbtree',scale_pos_weight=1,
                        random_over_sampler=False,use_best_iteration=False,
                        n_splits=5,is_boruta_sel=False,max_iter_boruta=100,
                        eval_metric='aucpr',
                        enable_categorical =False):
    params = { 'n_estimators':n_estimators,
              'max_leaves':25,
                'subsample':0.8,
              'random_state':145,
              'scale_pos_weight': scale_pos_weight,
              'booster': booster,
    #           'max_depth':6,
            'learning_rate':0.05,
             'colsample_bytree':0.6,#0.85,
             'lambda':0.05,
             'alpha':0.1,
             }
    
    
    if booster=='gblinear':
        params = { 'n_estimators':n_estimators,
                  'random_state':145,
                  'scale_pos_weight': scale_pos_weight,
                  'booster': booster,
                'learning_rate':1,#0.05,
#                  'lambda':0.05,
#                  'alpha':0.1,

                 }     
    
    params['enable_categorical']=enable_categorical
    if enable_categorical:
        params['tree_method']='approx'

    # xgb_model = xgb.XGBClassifier(**params)
    xgb_models = []
    scores  = []
    ratios =[]
    tr_cutoffs=[]
    val_cutoffs=[]
    tr_last_cutoffs=[]
    val_cutoff_percs=[]
    
    X = test_combined_imp[features_test].reset_index(drop=True)
    y = test_combined_imp['target'].reset_index(drop=True)
    
    top_ratios = []
    iterations = []
    first_match_positions= []
    best_iterations =[]
    sel_features_folds = []
    
    
    kfold = StratifiedKFold(n_splits=n_splits, random_state=756, shuffle=True)
    oof_probs = np.zeros(len(test_combined_imp))
    oversample = RandomOverSampler(sampling_strategy='minority',random_state=756)

    for fold,(tr_index, val_index) in enumerate(kfold.split(X, y)):
        X_tr,y_tr = X.iloc[tr_index],y.iloc[tr_index]
        X_val,y_val = X.iloc[val_index],y.iloc[val_index]
        
#         print(f'{val_index=}')

        # fit and apply the transform
        if random_over_sampler:
            X_tr, y_tr = oversample.fit_resample(X_tr, y_tr)
#         print(pd.Series(y_tr).value_counts())


        print(f'\n ******************* fold: {fold} ********')
#         print(f'tr size: { len(X_tr)}  val size:  {len(X_val)}')
        xgb_model = xgb.XGBClassifier(**params)
    
        
#         xgb_model.fit(X_tr,y_tr,
#                      verbose=0,
#                      eval_set = [(X_val,y_val)],
#                      eval_metric='aucpr',
# #                     eval_metric = cust_top_n_auc,
#                       early_stopping_rounds=50)
        
        sel_features = features_test.copy()
        if is_boruta_sel:
            params['n_estimators']=100
            xgb_model = xgb.XGBClassifier(**params)
            
            sel_features=get_boruta_selected_data(X_tr,y_tr,xgb_model,max_iter=max_iter_boruta)
            X_tr = X_tr[sel_features]
            X_val = X_val[sel_features]
            
            params['n_estimators']=n_estimators
        
        #fit model again after feature selection
        xgb_model.fit(X_tr,y_tr,
                     verbose=100,
                     eval_set = [(X_val,y_val)],
                     eval_metric=eval_metric,
#                     eval_metric = cust_top_n_auc,
                      early_stopping_rounds=None)        

        
        
        sel_features_folds.append(sel_features)
        xgb_models.append(xgb_model)
        
        best_iterations.append(xgb_model.get_booster().best_iteration)
        best_n_limit = xgb_model.get_booster().best_ntree_limit
        
        if use_best_iteration:
            val_probs = xgb_model.predict_proba(X_val,
                                               iteration_range=[0,best_n_limit]
#                                                iteration_range=[best_n_limit-1,best_n_limit]
                                               )[:,1]
        else:
            val_probs = xgb_model.predict_proba(X_val)[:,1]
            
        oof_probs[val_index] = val_probs
        
        val_success, val_ratio,val_top_prob_ind,match_pos_arr=get_top_success_count(val_probs,y_val.to_numpy(),
                                                                      top_n=top_n,asc=asc)
        if len(match_pos_arr)==0:
            first_match_pos=11
        else:
            first_match_pos = match_pos_arr[0]
        first_match_positions.append(first_match_pos)
        print('val match positions:',match_pos_arr)
        
#         print(f'total:{top_n} success:{val_success} top ratio:{val_ratio}')
        top_ratios.append(val_ratio)
        
        opt_cutoff, f1score = get_opt_cutoff_prec(y_val,val_probs)
#         print('valid cutoff:',opt_cutoff)
        val_cutoffs.append(opt_cutoff)
        opt_cutoff_perc = stats.percentileofscore(val_probs, opt_cutoff, 'weak') / 100
        val_cutoff_percs.append(opt_cutoff_perc)
        
    
        val_labels = convert_probtolabels(val_probs,cutoff=opt_cutoff) 
        mask1 = (val_labels==1)
        mask2 = (y_val==1)
        total = len(val_labels[mask1])
        success = len(val_labels[mask1 & mask2])
        ratio = success/total if total!=0 else 0
#         print(f'ratio: {ratio} total: {total} success: {success}')
        ratios.append(ratio)
        cur_f1 = f1_score(y_val, val_labels,average='macro')
        cur_acc = accuracy_score(y_val,val_labels)
#         print(f'val accuracy score:{cur_acc} f1 score:{cur_f1:.4f}')
        scores.append(cur_acc)
        
        dump_list = xgb_model.get_booster().get_dump()
        cur_iterations = len(dump_list)
        iterations.append(cur_iterations)
#         print('Iteration params:',cur_iterations,xgb_model.get_booster().best_iteration,
#               xgb_model.get_booster().best_ntree_limit)
        
#     print('\ntop ratios:',top_ratios)
    print()
#     print('ratios:',ratios)
#     print('scores:',scores)
#     print('val_cutoffs:',val_cutoffs)
    print('top ratio mean:',np.mean(top_ratios))
    print('first match pos mean:',np.mean(first_match_positions))
    print('ratio mean:',np.mean(ratios))
#     print('score mean:',np.mean(scores))
    mean_cutoff = np.mean(val_cutoffs)
#     print('cutoff mean:',mean_cutoff)
    mean_cutoff_perc= np.mean(val_cutoff_percs)
    
    print('iterations:',iterations)
    mean_iterations = np.mean(iterations)
#     print('iterations mean:',mean_iterations)
    mean_best_iterations = np.mean(best_iterations)
#     print('best iterations mean:',mean_best_iterations)
#         print(f'total:{top_n} success:{val_success} top ratio:{val_ratio}')

    #oof performance
    oof_success, oof_ratio,oof_top_prob_ind,oof_match_pos_arr=get_top_success_count(oof_probs,y.to_numpy(),
                                                                  top_n=top_n,asc=asc)

    print('oof match positions:',oof_match_pos_arr)
    print('oof top ratio:',oof_ratio)
    
    oof_average_precision = average_precision_score(y.to_numpy(),oof_probs)
    print('oof AUC PR Score:',oof_average_precision)
   
    return xgb_models,top_ratios,mean_iterations,mean_cutoff,mean_cutoff_perc,sel_features_folds,oof_match_pos_arr,oof_probs

def gen_top_mean_data_ml_final(test_combined_imp,features_test,
                            eval_set=None,early_stopping_rounds=None,eval_metric='aucpr',
                        booster = 'gbtree',scale_pos_weight=1,
                        n_estimators =100,random_over_sampler=False,
                        is_boruta_sel=False,max_iter_boruta=100,
                        enable_categorical =False):
    params = { 'n_estimators':n_estimators,
              'max_leaves':25,
                'subsample':0.8,
              'random_state':145,
              'scale_pos_weight': scale_pos_weight,
              'booster': booster,
    #           'max_depth':6,
            'learning_rate':0.05,
             'colsample_bytree':0.6,#0.85,
             'lambda':0.05,
             'alpha':0.1,
             }
    
    
    if booster=='gblinear':
        params = { 'n_estimators':n_estimators,
                  'random_state':145,
                  'scale_pos_weight': scale_pos_weight,
                  'booster': booster,
                'learning_rate':0.05,
                 'lambda':0.05,
                 'alpha':0.1,

                 }     
    
  
    params['enable_categorical']=enable_categorical
    if enable_categorical:
        params['tree_method']='approx'    
#     if eval_set is not None:
#         params['eval_metric']='aucpr'
#         params['early_stopping_rounds']=early_stopping_rounds
    
    
    X = test_combined_imp[features_test].reset_index(drop=True)
    y = test_combined_imp['target'].reset_index(drop=True)
    oversample = RandomOverSampler(sampling_strategy='minority',random_state=756)    
    if random_over_sampler:
        X , y = oversample.fit_resample(X , y)


    print(f'\n ******************* Finalized Model ********')
    xgb_model = xgb.XGBClassifier(**params)
#     xgb.set_params(**params)
    xgb_model.fit(X,y,eval_set = eval_set,eval_metric=eval_metric,verbose=0,
                  early_stopping_rounds=early_stopping_rounds)
    
    sel_features = features_test.copy()
    if is_boruta_sel:
        sel_features=get_boruta_selected_data(X,y,xgb_model,max_iter=max_iter_boruta)
        X = X[sel_features]
        xgb_model.fit(X,y)
        
    dump_list = xgb_model.get_booster().get_dump()
    cur_iterations = len(dump_list)
    best_iteration=xgb_model.get_booster().best_iteration
    print(f'Final cur iteration: {cur_iterations} best iteration:{best_iteration}')
    
    return xgb_model, sel_features,best_iteration

In [111]:
def filter_pos(oof_first_pos):
    default_valid = [1,2,3,4,5]
    if oof_first_pos > 5:
        return default_valid
    
    if oof_first_pos==1:
        return [1,2,3]
    elif oof_first_pos==2:
        return [1,2,3,4]
    elif oof_first_pos==3:
        return default_valid
    elif oof_first_pos==4:
        return [2,3,4,5]
    elif oof_first_pos==5:
        return [3,4,5]
    else:
        return default_valid
            
def filt_good_rolls(top_test,good_rolls):
    mask_good =None
    for col in good_rolls:
        if mask_good is None:
            mask_good = (top_test[col]==top_test[col].max())
        else:
            mask_good = mask_good |  (top_test[col]==top_test[col].max())
            
    cur_filt_ks= top_test[~mask_good]['k'].values
#     print(f'good roll_ks:{cur_filt_ks}')
    return cur_filt_ks
def filter_k(test_all_k,test_probs,all_k_imp,top_k_ele_imp,oof_first_pos,
             imp_df,imp_feats_count=1,top_n=5):
    test_all_k['probs_new']=test_probs
    top_test = test_all_k.sort_values('probs_new',ascending=False)[:5].reset_index(drop=True)
    
    roll_imp_feats = [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:imp_feats_count]
    #select only good rolls that is not having highest roll values in top 5
    good_rolls =   [
                    'roll_25',
                  'roll_25_nonce',
                 ] + roll_imp_feats
#     initial_rolls = [good_rolls[:3]]


    initial_rolls = good_rolls[3:4]
    
    
    
    cur_filt_ks= filt_good_rolls(top_test,initial_rolls)
    print(f'k after good rolls count {len(cur_filt_ks)} values: {cur_filt_ks} feats: {initial_rolls}')
    
    #remove top k belonging to hash important data
    if len(cur_filt_ks)>1:
        top_k_ele_flat = [x for sublist in top_k_ele_imp for x in sublist]
        cur_filt_ks = set(cur_filt_ks).difference(set(top_k_ele_flat))
        print(f'k after top k- count {len(cur_filt_ks)} values: {cur_filt_ks}')
        
    #select only good pos according to oof pos
    if len(cur_filt_ks)>1:
        good_pos_list= filter_pos(oof_first_pos)
        if len(good_pos_list)!=5:
            good_pos_ks= top_test[top_test.index.isin(good_pos_list)]['k']
            cur_filt_ks = set(cur_filt_ks).intersection(set(good_pos_ks))
            print(f'k after oof pos- count {len(cur_filt_ks)} values: {cur_filt_ks}')
        
    #remove highest k 
    if len(cur_filt_ks)>1:
        cur_filt_ks = set(cur_filt_ks).difference(set({top_test['k'].max()}))
        print(f'k after high k- count {len(cur_filt_ks)} values: {cur_filt_ks}')
        
        
    #select only all k belonging to hash important data
    if len(cur_filt_ks)>1:
        all_k_ele_flat = [x for sublist in all_k_imp for x in sublist]
        cur_filt_ks = set(cur_filt_ks).intersection(set(all_k_ele_flat))
        print(f'k after all k- count {len(cur_filt_ks)} values: {cur_filt_ks}')
        if len(cur_filt_ks)==0:
            print('WARNING: ALL K Filter removed all elements')
        
    for good_roll in [ col for col in good_rolls if col not in initial_rolls]:
        if len(cur_filt_ks)<=1:
            break
        good_filt_ks= filt_good_rolls(top_test,[good_roll])
        cur_filt_ks = set(cur_filt_ks).intersection(set(good_filt_ks))
        print(f'k after good rolls count {len(cur_filt_ks)} values: {cur_filt_ks} feats: {good_roll}')
        
    mask = create_target_mask(top_test)
    success_ks = top_test[mask]['k'].values
    matching_ks = set(cur_filt_ks).intersection(set(success_ks))
    print(f'\nManual Matched Ks: {matching_ks} Actual Success Ks: {success_ks} Filtered List: {cur_filt_ks}')
    match_success = len(matching_ks)!=0
    
    print(f'Success client seeds: {top_test[mask]["client_seed"].values}')
    
    print(f'Selected Client Seed(s):{top_test[top_test["k"].isin(cur_filt_ks)]["client_seed"].values}')
    print(f'Selected Roll Actual:{top_test[top_test["k"].isin(cur_filt_ks)]["roll_actual"].values}')
    
    return match_success,matching_ks,top_test

def filter_k_from_top(test_all_k,test_probs,all_k_imp,top_k_ele_imp,top_n=5):
    test_all_k['probs_new']=test_probs
    top_test = test_all_k.sort_values('probs_new',ascending=False)[:5]
    k_top_pos = top_test["k"].values
    top_test = top_test.sort_values('k',ascending=True).reset_index(drop=True)
    mask = create_target_mask(top_test)
    k_indices = top_test[mask]['k'].index
    k_matches = top_test[mask]["k"].values
    k_top = top_test["k"].values
    print(f'top k and its positions:{k_top_pos} ')
    print(f'k ranks: {k_indices} matching k: {k_matches}  top k:{k_top} ')
    all_k_ele_flat = [x for sublist in all_k_imp for x in sublist]
    top_k_ele_flat = [x for sublist in top_k_ele_imp for x in sublist]
    
    k_filt_hash = set(k_top).intersection(set(all_k_ele_flat))
    print(f'k existence in all hash imp- count {len(k_filt_hash)} values: {k_filt_hash}')
    k_filt_hash_top = set(k_top).intersection(set(top_k_ele_flat))
    print(f'k existence in top hash imp- count {len(k_filt_hash_top)} values: {k_filt_hash_top}')
    
def check_roll_exist_top(test_all_k,test_probs,imp_df,imp_feats_count=1,top_n=5):  
    test_all_k['probs_new']=test_probs
    top_test = test_all_k.sort_values('probs_new',ascending=False)[:5]
    roll_feats = ['roll_0','roll_1','roll_25','roll_50',
                  'roll_0_nonce','roll_1_nonce','roll_25_nonce','roll_50_nonce',
                  'roll_mean_25_50','roll_mean_25_50_nonce',
                  'roll_mean_25_50_nonce','roll_mean_nonce','roll_mean',
                 'count_hi', 'count_hi_1_60','count_hi_nonce', 'count_hi_1_60_nonce']
    
    roll_imp_feats = [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:imp_feats_count]
#     roll_feats +=list(set(roll_imp_feats).difference(set(roll_feats)))
    
    mask = create_target_mask(top_test)
    top_match = top_test[mask]
    high_collision=[]
    high_collision_imp=[]
    
    if len(top_match)!=0:
        for col in roll_feats:
            top_match_filt = top_match[top_match[col]==top_test[col].max()]
            if len(top_match_filt)!=0:
                high_collision.append(col)
        for i,col in enumerate(roll_imp_feats):
            top_match_filt = top_match[top_match[col]==top_test[col].max()]
            if len(top_match_filt)!=0:
                high_collision_imp.append(f'imp_feat_{i+1}')
                
#     good_rolls = ['roll_25','roll_25_nonce',roll_imp_feats[0],roll_imp_feats[1]]
    good_rolls = ['roll_25','roll_25_nonce',roll_imp_feats[0]]
    mask_good =None
    for col in good_rolls:
        if mask_good is None:
            mask_good = (top_test[col]==top_test[col].max())
        else:
            mask_good = mask_good |  (top_test[col]==top_test[col].max())
            
    high_roll_ks= top_test[~mask_good]['k'].values
    print(f'high_roll_ks:{high_roll_ks}')
            
    print(f'High Collisions Count: {len(high_collision)} Values :{high_collision}')
    print(f'High Collisions Imp Count: {len(high_collision_imp)} Values :{high_collision_imp} ')
    print(f'All Imp Feats: {roll_imp_feats} ')
            
    return high_collision,high_collision_imp

In [112]:
def display_model_output(test_probs, y_test, top_n=100,asc=False,is_print=True):
    average_precision = average_precision_score(y_test, test_probs)
    if is_print:
        print(f'AUC PR Score:{average_precision}')
    test_success, test_ratio,test_top_prob_ind,match_pos_arr=get_top_success_count(test_probs,y_test,
                                                                      top_n=top_n,asc=asc)
    if is_print:
        print(f'total:{top_n} success:{test_success} top ratio:{test_ratio} match positions: {match_pos_arr}')
    return match_pos_arr, average_precision
    

In [113]:
# mask = create_target_mask(test_all_k)        

# test_all_k['target']=0
# test_all_k.loc[mask,'target']=1

# y_test = test_all_k['target']
# test_probs= np.zeros(len(test_all_k))
# for model in xgb_models:
#     test_probs += (model.predict_proba(test_all_k[features_test])[:,1]) 
# test_probs = test_probs / len(xgb_models)

# display_model_output(test_probs, y_test.to_numpy(), top_n=100,asc=False)

In [114]:
# %%time

# use_best_iteration=False
# is_rank=False
# is_rank_pct=False
# is_individual_feats = False

# save_models=False
# load_models=False
# eval_metric = 'aucpr'
        
# cur_feats_count = 2
# top_order_asc = False
# top_n=100
# random_over_sampler=False
# n_splits=5

# is_boruta_sel=False
# max_iter_boruta=25

# rand_count_fetch=10
# rand_iterations=1000

# if is_individual_feats:
#     option = 'indl'
# elif is_rank:
#     option = 'rank'
# else:
#     option = 'wo_rank'
    
# if is_rank_pct:
#     option += '_pct'
# if random_over_sampler:
#     option += '_oversampler'  
    
    
# # option += '_20feats'    
# option += '_5feats_roll1000' 

In [115]:
# # test_all_k['rand_feat'] =  test_all_k['roll_25_nonce'].copy() 
# rand_all = None
# for i in range(len(test_all_k_imp)):
#     test_all_k_imp[i]['rand_feat'] = np.random.randint(1,200,size=len(test_all_k_imp[i]))
#     values= test_all_k_imp[i]['rand_feat'].unique()
#     if rand_all is None:
#         rand_all = values
#     else:
#         rand_all=np.concatenate((rand_all,values))

# rand_all = np.unique(rand_all)        
# # test_all_k['rand_feat'] =  np.random.randint(1,100,size=len(test_all_k))
# rand_all = np.random.choice(rand_all,size=len(test_all_k))
# test_all_k['rand_feat'] = rand_all

In [400]:
# test_all_k['rand_feat'] =  test_all_k['roll_25_nonce'].copy() 
for i in range(len(test_all_k_imp)):
    test_all_k_imp[i]['rand_feat'] = np.random.randint(1,100,size=len(test_all_k_imp[i]))

test_all_k['rand_feat'] =  np.random.randint(1,100,size=len(test_all_k))

In [401]:
test_all_k['rand_feat']=test_all_k['rand_feat'].astype('category')
for i in range(len(test_all_k_imp)):
    test_all_k_imp[i]['rand_feat'] = test_all_k_imp[i]['rand_feat'].astype('category') 

In [402]:
# one = set(test_all_k['rand_feat'].unique())
# two = set(test_all_k_imp[1]['rand_feat'].unique())
# print(len(one),len(two))
# print(one.difference(two))

In [403]:
LOW_TARGET = 1000
HIGH_ANALYSIS =False

In [404]:
%%time

key_feats = ['rand_feat']

use_best_iteration=False
enable_categorical = True

save_models=False
load_models=False
eval_metric = 'aucpr'

scale_pos_weight=1
booster='gbtree' #gblinear, gbtree
        
cur_feats_count = 2
top_order_asc = False
top_n=10
n_splits=5

is_boruta_sel=False
max_iter_boruta=25

rand_count_fetch=10
rand_iterations=1000

model_iterations = 100

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 14.1 µs


In [405]:
%%time 

conditions = [(False,False,False,False)]

# conditions = [(False,False,False,False),(True,False,False,False),(True,True,False,False),
#               (True,True,True,False),(True,False,True,False),
#              (False,False,False,True),(True,False,False,True),(True,True,False,True),
#               (True,True,True,True),(True,False,True,True)]

for condn in conditions:

    print(f'\n****************** NONCE {nonce}  ****************** ')

    is_rank=condn[0]
    is_rank_pct=condn[1]
    is_individual_feats = condn[2]
    random_over_sampler=condn[3]

    print(f'************* "{booster} condition sampler":{random_over_sampler}  "rank":{is_rank}  "pct":{is_rank_pct}  "indl":{is_individual_feats} *************')


    if is_individual_feats:
        option = 'indl'
    elif is_rank:
        option = 'rank'
    else:
        option = 'wo_rank'

    if is_rank_pct:
        option += '_pct'
    if random_over_sampler:
        option += '_oversampler'  


    # option += '_20feats'    
    option += '_5feats_roll1000_nonce' 


    cur_feats_count=140

    if is_individual_feats:
        for cur_hash_imp in test_all_k_imp:
            cur_hash_imp= get_sim_summary(cur_hash_imp,
                                           True,cur_feats_count,is_rank_pct=is_rank_pct)

    mask = create_target_mask(test_all_k)        

    test_all_k['target']=0
    test_all_k.loc[mask,'target']=1

    y_test = test_all_k['target']



    test_combined_imp=pd.concat(test_all_k_imp,axis=0)
    mask = create_target_mask(test_combined_imp)  
    test_combined_imp['target']=0
    test_combined_imp.loc[mask,'target']=1
    
#     test_combined_imp = test_combined_imp.sample(n=50)

    print(len(test_all_k))
    print(len(test_combined_imp))
    
    # cur_feats_count=140

    if not is_individual_feats:
        test_combined_imp= get_sim_summary(test_combined_imp,
                                             True,cur_feats_count,is_rank_pct=is_rank_pct)

#     cur_features =  features_test.copy()
    
    cur_features =  key_feats.copy()
    
#     cur_features = [col for col in features_test if '_nonce' in col]
    print(len(cur_features))
    # imp_roll_feats = [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:cur_feats_count]
    # cur_features = [col for col in cur_features if not(col.startswith('roll_')) or (col in imp_roll_feats)]
    # print(cur_features)
    if is_rank:
        cur_features += ['ratio_mean_rank','roll_mean_rank','ratio_group_roll_rank']            


    if load_models:
        model_set = joblib.load(f'data/models/imp_models_{nonce}_{option}_{file_pattern_str}_pattern.dump')
        cur_models = model_set[2]
        oof_match_pos_arr,mean_iterations,best_iteration_val=model_set[3]
        print(f'OOF Match Positions:{oof_match_pos_arr}')
    else:
        cur_models,top_ratios,mean_iterations,mean_cutoff,\
        mean_cutoff_perc,sel_features_folds,oof_match_pos_arr,oof_probs = \
                                                    gen_top_mean_data_ml(test_combined_imp,
                                                                           cur_features,
                                                                             booster = booster,
                                                                            scale_pos_weight=scale_pos_weight,
                                                                            n_estimators=model_iterations,
                                                                             eval_metric=eval_metric,
                                                                             asc=top_order_asc,top_n=top_n,
                                                                            random_over_sampler=random_over_sampler,
                                                                        use_best_iteration=use_best_iteration,
                                                                        n_splits=n_splits,
                                                                        is_boruta_sel=is_boruta_sel,
                                                                         max_iter_boruta=max_iter_boruta,
                                                                        enable_categorical=enable_categorical)



    test_all_k= get_sim_summary(test_all_k,
                                True,cur_feats_count,is_rank_pct=is_rank_pct)

    test_probs= np.zeros(len(test_all_k))


    best_iterations=[]
    for model in cur_models:
        test_probs += (model.predict_proba(test_all_k[cur_features])[:,1]) 
    test_probs = test_probs / len(cur_models)
    
    test_probs_avg = test_probs.copy()

    display_model_output(test_probs, y_test.to_numpy(), top_n=top_n,asc=top_order_asc)

#     oof_first_pos =   oof_match_pos_arr[0]  

    # filter_success,filter_ks,top_test = filter_k(test_all_k,test_probs,all_k_imp,top_k_ele_imp,oof_first_pos,
    #          imp_df,imp_feats_count=10,top_n=5)

    # filter_k_from_top(test_all_k,test_probs,all_k_imp,top_k_ele_imp,top_n=5)
    # high_collision_result = check_roll_exist_top(test_all_k,test_probs,imp_df,imp_feats_count=5,top_n=5)

    #Final Model

    print(f'mean iterations:{mean_iterations}')
    if use_best_iteration:
        n_estimators = 100 #int(np.mean(best_iterations))
    else:
        n_estimators = int(mean_iterations)

    print('\n------------------ Finalized Model ---------------------')
    if load_models:
        model_final = model_set[0]
        model_final_val = model_set[1]
    #     print(f'*************** First Run of final model with kfold mean iterations {mean_iterations} ***************')
    #     model_final,sel_features_final,best_iteration = gen_top_mean_data_ml_final(test_combined_imp,cur_features,
    #                             n_estimators = n_estimators,random_over_sampler=random_over_sampler,
    #                             is_boruta_sel=is_boruta_sel,
    #                             eval_metric=eval_metric,                                                       
    #                             eval_set = [(test_combined_imp[cur_features],test_combined_imp['target'].to_numpy())],
    #                             early_stopping_rounds = 50,
    #                              max_iter_boruta=max_iter_boruta)
        print(f'*************** Second Run of final model with actual as eval set ***************')
        model_final_val,sel_features_final_val,best_iteration_val = gen_top_mean_data_ml_final(test_combined_imp,cur_features,
                                eval_set = [(test_all_k[cur_features],y_test)],
                                early_stopping_rounds = 50,eval_metric=eval_metric,booster=booster,
                                n_estimators = model_iterations,random_over_sampler=random_over_sampler,
                                is_boruta_sel=is_boruta_sel,
                                 max_iter_boruta=max_iter_boruta)
    else:
        print(f'*************** First Run of final model with kfold mean iterations {mean_iterations} ***************')
        model_final,sel_features_final,best_iteration = gen_top_mean_data_ml_final(test_combined_imp,cur_features,
                                n_estimators = n_estimators,random_over_sampler=random_over_sampler,
                                eval_metric=eval_metric,booster=booster,
                                is_boruta_sel=is_boruta_sel,
                                 max_iter_boruta=max_iter_boruta,
                                enable_categorical=enable_categorical)

        model_final_val = model_final
        best_iteration_val = mean_iterations

    #     print(f'*************** Second Run of final model with actual as eval set ***************')
    #     model_final_val,sel_features_final_val,best_iteration_val = gen_top_mean_data_ml_final(test_combined_imp,cur_features,
    #                             eval_set = [(test_all_k[cur_features],y_test)],
    #                             early_stopping_rounds = 50,eval_metric=eval_metric,
    #                             n_estimators = 1000,random_over_sampler=random_over_sampler,
    #                             is_boruta_sel=is_boruta_sel,
    #                              max_iter_boruta=max_iter_boruta)

    print(f'*************** First Run of final model with kfold mean iterations {mean_iterations} ***************')
    test_probs = (model_final.predict_proba(test_all_k[cur_features])[:,1]) 
    display_model_output(test_probs, y_test.to_numpy(), top_n=top_n,asc=top_order_asc)
    print(f'*************** Second Run of final model with actual as eval set ***************')
    test_probs = (model_final_val.predict_proba(test_all_k[cur_features])[:,1]) 
    display_model_output(test_probs, y_test.to_numpy(), top_n=top_n,asc=top_order_asc)

    if save_models:
        joblib.dump([model_final,model_final_val,cur_models,
                     [oof_match_pos_arr,mean_iterations,best_iteration_val]],
                    f'data/models/imp_models_{nonce}_{option}_{file_pattern_str}_pattern.dump')


# filter_success,filter_ks,top_test_final = filter_k(test_all_k,test_probs,all_k_imp,top_k_ele_imp,oof_first_pos,
#          imp_df,imp_feats_count=10,top_n=5)

# filter_k_from_top(test_all_k,test_probs,all_k_imp,top_k_ele_imp,top_n=5)
# high_collision_result = check_roll_exist_top(test_all_k,test_probs,imp_df,imp_feats_count=5,top_n=5)



****************** NONCE 2826  ****************** 
************* "gbtree condition sampler":False  "rank":False  "pct":False  "indl":False *************
1000
5000
1

 ******************* fold: 0 ********
[0]	validation_0-aucpr:0.09485
[99]	validation_0-aucpr:0.09238
val match positions: []

 ******************* fold: 1 ********
[0]	validation_0-aucpr:0.09069
[99]	validation_0-aucpr:0.08979
val match positions: []

 ******************* fold: 2 ********
[0]	validation_0-aucpr:0.09472
[99]	validation_0-aucpr:0.10034
val match positions: [2]

 ******************* fold: 3 ********
[0]	validation_0-aucpr:0.09376
[99]	validation_0-aucpr:0.09583
val match positions: []

 ******************* fold: 4 ********
[0]	validation_0-aucpr:0.09948
[99]	validation_0-aucpr:0.09389
val match positions: []

top ratio mean: 0.02
first match pos mean: 9.2
ratio mean: 0.10348672997321103
iterations: [100, 100, 100, 100, 100]
oof match positions: []
oof top ratio: 0.0
oof AUC PR Score: 0.09428384842577814
AUC 

In [422]:
def get_random_client_seed(test):
    size = len(test)
    final_test= test.sample(n=size)
    rand_pos=random.randint(0,size-1)
    sample= final_test.iloc[rand_pos]
#     sample=final_test.reset_index(drop=True).loc[rand_pos]
#     rand_pos = sample.index[0]
    print(f'rand_pos:{rand_pos}')
    return sample['client_seed'],sample,final_test,rand_pos

In [ ]:
# test_probs= np.zeros(len(test_all_k))
# for model in cur_models:
#     cur_probs =model.predict_proba(test_all_k[cur_features])[:,1] 
#     np.random.shuffle(cur_probs) 
#     test_probs += cur_probs
# test_probs = test_probs / len(cur_models)
# display_model_output(test_probs, y_test.to_numpy(), top_n=100,asc=False)
# # display_model_output(test_probs, y_test.to_numpy(), top_n=100,asc=True)

In [ ]:
# LOW_TARGET = 4750
# HIGH_TARGET= 5250

# HIGH_ANALYSIS =False

In [ ]:
%%time
count_top_k = 0
trials = 50
k=5
top_data_all = None 
asc_cur =False
for i in range(trials):
    temp= test_all_k.sample(frac=0.1)
    y_temp = temp['target'].to_numpy()
    test_probs= np.zeros(len(temp))
    for model in cur_models:
        test_probs += (model.predict_proba(temp[cur_features])[:,1]) 
    test_probs = test_probs / len(cur_models)
    match_pos_arr, average_precision = display_model_output(test_probs, y_temp, top_n=k,asc=asc_cur,
                                                           is_print=False)
    count_top_k +=  len(match_pos_arr[match_pos_arr<=k]) / k
    
#     top_data_all = temp.sort_values('probs_new',ascending=False)[:k].iloc[match_pos_arr-1]
    temp['probs_new']= test_probs
    top_data_cur =temp.sort_values('probs_new',ascending=asc_cur)[:k]
    if top_data_all is None:
        top_data_all = top_data_cur.copy()
    else:
        top_data_all = pd.concat([top_data_all,top_data_cur],axis=0)
# print(f'\ntotal top 5:{count_top_k} ratio:{(count_top_k/trials)/k}')
print(f'\ntotal top 5:{count_top_k} ratio:{(count_top_k/trials)}')

# top_data_all= top_data_all.drop_duplicates()

total = len(top_data_all)
mask= create_target_mask(top_data_all)
success = len(top_data_all[mask])
ratio = success / total
print(f'ratio:{ratio} success:{success} total:{total}')

# display_model_output(test_probs, y_temp, top_n=100,asc=True)

In [431]:
%%time
rand_seed = 488
count_top_k = 0
trials =500
k=5
top_data_all = None 
asc_cur =False

test_probs= np.zeros(len(test_all_k))
for model in cur_models:
    test_probs += (model.predict_proba(test_all_k[cur_features])[:,1]) 
test_probs = test_probs / len(cur_models)
test_all_k['probs_new']=test_probs

for i in range(trials):
    temp= test_all_k.sample(n=10,random_state=rand_seed+i)
    y_temp = temp['target'].to_numpy()
#     match_pos_arr, average_precision = display_model_output(test_probs,
#                                                             y_temp, top_n=k,asc=asc_cur,
#                                                            is_print=False)
#     count_top_k +=  len(match_pos_arr[match_pos_arr<=k]) / k
    
    
#     top_data_all = temp.sort_values('probs_new',ascending=False)[:k].iloc[match_pos_arr-1]
    top_data_cur =temp.sort_values('probs_new',ascending=asc_cur)[:k]
#     top_data_cur = top_data_cur[(top_data_cur['roll_25_nonce']==top_data_cur['roll_25_nonce'].min())
#                                & (top_data_cur['roll_50_nonce']==top_data_cur['roll_50_nonce'].min())]
    top_data_cur = top_data_cur[(top_data_cur['roll_25_nonce']==top_data_cur['roll_25_nonce'].max())
                               & (top_data_cur['roll_50_nonce']==top_data_cur['roll_50_nonce'].max())
#                                & (top_data_cur['roll_25']==top_data_cur['roll_25'].max())
#                                & (top_data_cur['roll_50']==top_data_cur['roll_50'].max())
                               & (top_data_cur['probs_new']==top_data_cur['probs_new'].max())
                               ]
#     top_data_cur = top_data_cur[(top_data_cur['roll_25_nonce']==top_data_cur['roll_25_nonce'].max())
#                                & (top_data_cur['roll_50_nonce']==top_data_cur['roll_50_nonce'].max())]
#     top_data_cur = top_data_cur[(top_data_cur['roll_50_nonce']==top_data_cur['roll_50_nonce'].max())]
#     top_data_cur = top_data_cur[(top_data_cur['roll_25_nonce']==top_data_cur['roll_25_nonce'].max())]
#     top_data_cur = top_data_cur.sort_values('roll_25_nonce',ascending=False)[:1]
#     top_data_cur = top_data_cur.sort_values('roll_25',ascending=True)[:3]
#     top_data_cur = top_data_cur.sort_values('roll_50_nonce',ascending=True)[:2]
#     top_data_cur = top_data_cur.sort_values('roll_50',ascending=True)[:1]

    if top_data_all is None:
        top_data_all = top_data_cur.copy()
    else:
        top_data_all = pd.concat([top_data_all,top_data_cur],axis=0)
# print(f'\ntotal top 5:{count_top_k} ratio:{(count_top_k/trials)/k}')
print(f'\ntotal top 5:{count_top_k} ratio:{(count_top_k/trials)}')

# top_data_all= top_data_all.drop_duplicates()
total = len(top_data_all)
if total==0:
    success=0
    ratio=0
else:
    mask= create_target_mask(top_data_all)
    success = len(top_data_all[mask])
    ratio = success / total
print(f'ratio:{ratio} success:{success} total:{total}')

# display_model_output(test_probs, y_temp, top_n=100,asc=True)


total top 5:0 ratio:0.0
ratio:0.28 success:7 total:25
CPU times: user 8.8 s, sys: 96 ms, total: 8.9 s
Wall time: 8.79 s


In [432]:
match_pos_arr, average_precision = display_model_output(top_data_all['probs_new'].to_numpy(),
                                                        top_data_all['target'].to_numpy(), 
                                                        top_n=len(top_data_all),asc=False,
                                                           is_print=True)

# cutoff = int(len(top_data_all) * 25 /100)
# top_data_sorted = top_data_all.sort_values('probs_new',ascending=False)
# top_data_reduced = top_data_sorted.iloc[cutoff:len(top_data_sorted)-cutoff]
# print(len(top_data_reduced))

# match_pos_arr, average_precision = display_model_output(top_data_reduced['probs_new'].to_numpy(),
#                                                         top_data_reduced['target'].to_numpy(), 
#                                                         top_n=len(top_data_reduced),asc=False,
#                                                            is_print=True)

AUC PR Score:0.47409247757073847
total:25 success:7 top ratio:0.28 match positions: [ 1  3  4  6  7 23 25]


In [433]:
client_seed,rand_selected,final_test,rand_pos = get_random_client_seed(top_data_all)
print('Client Seed Selected')
print(client_seed)
final_test.iloc[rand_pos:rand_pos+1]

rand_pos:22
Client Seed Selected
woxpwoxpwoxpwoxpwoxpd1ca48835ae9685453877f4f86003af5e01248470b7fa645a5ad7bd3f5569145


,roll_actual,seed,hash,client_seed,client_index,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_non

In [336]:
print(len(top_data_all))
top_data_all[['roll_actual','target','probs_new']]

10


,roll_actual,target,probs_new
854,1762,0,0.195312
615,5589,0,0.159795
232,6118,0,0.138341
135,288,1,0.176766
909,4639,0,0.161484
996,6514,0,0.223588
745,5149,0,0.223588
561,153,1,0.174497
362,7259,0,0.223588
204,5302,0,0.229767


In [228]:
top_data_all.index

Index([646, 9, 373, 183, 488, 314, 643, 489, 711, 561, 646], dtype='int64')

In [302]:
%%time

rand_seed = 524
count_top_k = 0
trials = 1000
k=5
top_data_all = None 
asc_cur =False
test_probs= np.zeros(len(test_all_k))
for model in cur_models:
    test_probs += (model.predict_proba(test_all_k[cur_features])[:,1]) 
test_probs = test_probs / len(cur_models)
test_all_k['probs_new']=test_probs
index_list =[]

max_25_nonce_list = []
max_50_nonce_list = []
prob_list = []

for i in range(trials):
    temp= test_all_k.sample(n=10,random_state=rand_seed+i)

#     match_pos_arr, average_precision = display_model_output(test_probs,
#                                                             y_temp, top_n=k,asc=asc_cur,
#                                                            is_print=False)
    
    
    top_data_cur =temp.sort_values('probs_new',ascending=asc_cur)[:k]
    max_25_nonce_list.append(top_data_cur['roll_25_nonce'].max())
    max_50_nonce_list.append(top_data_cur['roll_50_nonce'].max())
    prob_list.append(top_data_cur['probs_new'].max())
    index_list.append(list(top_data_cur.index))

CPU times: user 6.56 s, sys: 100 ms, total: 6.66 s
Wall time: 6.62 s


In [303]:
%%time
print(len(index_list))    
mask = None
for indices_cur,max_25_nonce,max_50_nonce,max_prob in zip(index_list,max_25_nonce_list,
                                                          max_50_nonce_list,prob_list):
    mask1 = (test_all_k.index.isin(indices_cur))
    mask2 = (test_all_k['roll_25_nonce']==max_25_nonce) \
              & (test_all_k['roll_50_nonce']==max_50_nonce) \
              & (test_all_k['probs_new']==max_prob) 
    mask3 = mask1 & mask2
    mask = (mask | mask3) if mask is not None else mask3
    

top_data_all = test_all_k[mask]    

# top_data_all= top_data_all.drop_duplicates()
total = len(top_data_all)
if total==0:
    success=0
    ratio=0
else:
    mask_target= create_target_mask(top_data_all)
    success = len(top_data_all[mask_target])
    ratio = success / total
print(f'ratio:{ratio} success:{success} total:{total}')

# display_model_output(test_probs, y_temp, top_n=100,asc=True)

1000
ratio:0.06896551724137931 success:2 total:29
CPU times: user 636 ms, sys: 0 ns, total: 636 ms
Wall time: 635 ms


In [308]:
print(len(top_data_all))

29


In [312]:
index_list_flat = [item for row in index_list for item in row]
print(len(index_list_flat))

index_list_final = [ele for ele in index_list_flat if ele in list(top_data_all.index)]
print(len(index_list_final))
df = top_data_all.copy()

5000
310


In [313]:
len(set(index_list_final))

29

In [322]:
%%time
d = {k:test_all_k.index[test_all_k.index.values == k] for k, v in Counter(index_list_flat).items()}
idx = [y for x in index_list_flat for y in d[x]]
# df_new = test_all_k.loc[idx].reset_index(drop=True)
df_new = test_all_k.loc[idx]
print(len(df_new))

5000
CPU times: user 27.9 ms, sys: 11.9 ms, total: 39.9 ms
Wall time: 30.8 ms


In [320]:
df_new.head()

,roll_actual,seed,hash,client_seed,client_index,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_non

In [323]:
%%time
print(len(index_list))    
mask = None
for indices_cur,max_25_nonce,max_50_nonce,max_prob in zip(index_list,max_25_nonce_list,
                                                          max_50_nonce_list,prob_list):
    mask1 = (df_new.index.isin(indices_cur))
    mask2 = (df_new['roll_25_nonce']==max_25_nonce) \
              & (df_new['roll_50_nonce']==max_50_nonce) \
              & (df_new['probs_new']==max_prob) 
    mask3 = mask1 & mask2
    mask = (mask | mask3) if mask is not None else mask3
    

temp_data_all = df_new[mask]    

# top_data_all= top_data_all.drop_duplicates()
total = len(temp_data_all)
if total==0:
    success=0
    ratio=0
else:
    mask_target= create_target_mask(temp_data_all)
    success = len(temp_data_all[mask_target])
    ratio = success / total
print(f'ratio:{ratio} success:{success} total:{total}')

# display_model_output(test_probs, y_temp, top_n=100,asc=True)

1000
ratio:0.08064516129032258 success:25 total:310
CPU times: user 688 ms, sys: 3.9 ms, total: 692 ms
Wall time: 690 ms


In [300]:
thislist = ["apple", "banana", "cherry","banana","oranges","cherry"]
thislist= [ele for ele in thislist if ele not in ["oranges","banana"]]
thislist

['apple', 'cherry', 'cherry']

In [231]:
top_data_all.index

Index([9, 183, 314, 373, 488, 489, 561, 643, 646, 711], dtype='int64')

In [ ]:
Index([646, 9, 373, 183, 488, 314, 643, 489, 711, 561, 646], dtype='int64')


In [174]:
print(len(mask1))
print(len(test_all_k[mask1]))

1000
2


In [168]:
test_all_k[test_all_k.index.isin(indices_cur)]

,roll_actual,seed,hash,client_seed,client_index,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_non

In [167]:
top_data_all

,roll_actual,seed,hash,client_seed,client_index,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_non

In [129]:
from joblib import Parallel, delayed

In [183]:
%%time

def run_trial_single(data,models,asc=False,k=5):
    temp= data.sample(n=10)
    top_data_cur =temp.sort_values('probs_new',ascending=asc_cur)[:k]
    max_25_nonce = temp['roll_25_nonce'].max()
    index_cur = list(top_data_cur.index)
    return index_cur,max_25_nonce
 
# def run_trial_single(data,models,asc=False,k=5):
#     temp= data.sample(n=10)
#     y_temp = temp['target'].to_numpy()
#     test_probs= np.zeros(len(temp))
#     for model in models:
#         test_probs += (model.predict_proba(temp[cur_features])[:,1]) 
#     test_probs = test_probs / len(cur_models)
#     match_pos_arr, average_precision = display_model_output(test_probs,
#                                                             y_temp, top_n=k,asc=asc_cur,
#                                                            is_print=False)
    
#     temp['probs_new']= test_probs
#     top_data_cur =temp.sort_values('probs_new',ascending=asc_cur)[:k]
#     top_data_cur = top_data_cur[(top_data_cur['roll_25_nonce']==top_data_cur['roll_25_nonce'].max())]
# #     top_data_cur = top_data_cur[(top_data_cur['roll_25_nonce']==top_data_cur['roll_25_nonce'].max())
# #                                & (top_data_cur['roll_50_nonce']==top_data_cur['roll_50_nonce'].max())
# #                                & (top_data_cur['probs_new']==top_data_cur['probs_new'].max())
# #                                ]
#     return top_data_cur

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 11.2 µs


In [191]:
%%time
count_top_k = 0
trials = 1000
k=5
top_data_all = None 
asc_cur =False
test_probs= np.zeros(len(test_all_k))
for model in cur_models:
    test_probs += (model.predict_proba(test_all_k[cur_features])[:,1]) 
test_probs = test_probs / len(cur_models)
test_all_k['probs_new']=test_probs
index_list =[]

delayed_calls = [delayed(run_trial_single)(test_all_k,cur_models,asc=asc_cur,k=k) for i in range(trials)]
results_delayed = Parallel(n_jobs=15,prefer='processes')(delayed_calls)
print('completed')

index_list = [t[0] for t in results_delayed]
max_25_nonce_list = [t[1] for t in results_delayed]

print(len(index_list))    
mask = None
for indices_cur,max_25_nonce in zip(index_list,max_25_nonce_list):
#     mask1 = (test_all_k['roll_25_nonce']==top_data_cur['roll_25_nonce'].max()) \
#                                & (top_data_cur['roll_50_nonce']==top_data_cur['roll_50_nonce'].max()) \
#                                & (top_data_cur['probs_new']==top_data_cur['probs_new'].max()) 
    mask1 = (test_all_k.index.isin(indices_cur))
    mask2 = (test_all_k['roll_25_nonce']==max_25_nonce) 
    mask = (mask | mask2) if mask is not None else mask2
    

top_data_all = test_all_k[mask]    

# top_data_all= top_data_all.drop_duplicates()
total = len(top_data_all)
if total==0:
    success=0
    ratio=0
else:
    mask_target= create_target_mask(top_data_all)
    success = len(top_data_all[mask_target])
    ratio = success / total
print(f'ratio:{ratio} success:{success} total:{total}')

# display_model_output(test_probs, y_temp, top_n=100,asc=True)

completed
1000
ratio:0.5490909090909091 success:151 total:275
CPU times: user 15.6 s, sys: 2.43 s, total: 18 s
Wall time: 11.3 s


In [186]:
results_delayed[:5]

[([413, 383, 430, 890, 422], 9181),
 ([58, 128, 837, 255, 593], 9871),
 ([629, 421, 683, 550, 674], 8062),
 ([906, 485, 744, 841, 710], 9471),
 ([365, 715, 920, 189, 565], 9113)]

In [152]:
%%time
trials = 50
k=5
top_data_all = None 
asc_cur =False

delayed_calls = [delayed(run_trial_single)(test_all_k,cur_models,asc=asc_cur,k=k) for i in range(trials)]
results_delayed = Parallel(n_jobs=15,prefer='processes')(delayed_calls)

top_data_all = pd.concat(results_delayed,axis=0)

# top_data_all= top_data_all.drop_duplicates()
total = len(top_data_all)
if total==0:
    success=0
    ratio=0
else:
    mask= create_target_mask(top_data_all)
    success = len(top_data_all[mask])
    ratio = success / total
print(f'ratio:{ratio} success:{success} total:{total}')

# display_model_output(test_probs, y_temp, top_n=100,asc=True)

ratio:0.56 success:28 total:50
CPU times: user 1.92 s, sys: 286 ms, total: 2.2 s
Wall time: 1.53 s


In [ ]:
top_data_unique= top_data_all.drop_duplicates()

total = len(top_data_unique)
mask= create_target_mask(top_data_unique)
success = len(top_data_unique[mask])
ratio = success / total
print(f'ratio:{ratio} success:{success} total:{total}')

# display_model_output(test_probs, y_temp, top_n=100,asc=True)

In [ ]:
display_model_output(oof_probs, test_combined_imp['target'].to_numpy(), top_n=top_n,asc=True)

In [ ]:
# test_probs_linear = test_probs_avg.copy()
# test_probs_linear_final = test_probs.copy()

In [ ]:
test_all_k['probs_new']=test_probs_avg

top_data = test_all_k.sort_values('probs_new',ascending=False)[:5]
key_cols = ['roll_actual','roll_0','roll_1','probs_new','client_seed']
other_cols = [col for col in top_data.columns if col not in key_cols]
top_data[key_cols+other_cols]

In [ ]:
sel_idx  = top_data[top_data['roll_actual']==top_data['roll_actual'].max()].index[0]
sel_pos  = top_data.reset_index()[top_data.reset_index()['index']==sel_idx].index[0]+1
print(f'selected idx: {sel_idx} selected pos:{sel_pos}')

test_sel = top_data.loc[[sel_idx]]
# test_sel = top_data.iloc[sel_pos-1:sel_pos]
print('sel client seed:',test_sel['client_seed'].values[0])
test_sel[key_cols+other_cols]

In [ ]:
prev_seed = actual_seed

In [ ]:
# actual_seed='5fb04950c9347ad2c512377d45270b9756fa29a8350260451d470bb254d87907'
actual_seed='6a1f54767d9684db9851a372336906d00da9fe80a756b7718a76d959d896100c'
# actual_seed='5052b5bcae33b649af745cfd4bcee923e41d607ea6cfccffe899ad807d008a8f'
# Vectorize the function
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(actual_seed,
                                       top_data['client_seed'],
                                       nonce)
top_data['roll_actual_dummy']=top_data['roll_actual'].copy()
top_data['roll_actual']=roll_array
# final_test.to_csv(f'data/analysis/final_test_{nonce}.csv')

In [ ]:
top_data[key_cols+other_cols]

In [ ]:
# actual_seed='3d600138dff42a723846e3db6afa1af05fb6ecfa461b9c5a995ee49e941189bc'
# Vectorize the function
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(actual_seed,
                                       test_all_k['client_seed'],
                                       nonce)
test_all_k['roll_actual']=roll_array
# final_test.to_csv(f'data/analysis/final_test_{nonce}.csv')

In [ ]:
mask = create_target_mask(test_all_k)        
test_all_k['target']=0
test_all_k.loc[mask,'target']=1

y_test = test_all_k['target']

In [ ]:
test_probs= np.zeros(len(test_all_k))
for model in cur_models:
    test_probs += (model.predict_proba(test_all_k[cur_features])[:,1]) 
test_probs = test_probs / len(cur_models)
display_model_output(test_probs, y_test.to_numpy(), top_n=100,asc=False)
display_model_output(test_probs, y_test.to_numpy(), top_n=100,asc=True)

In [ ]:
test_probs = (model_final.predict_proba(test_all_k[cur_features])[:,1]) 
display_model_output(test_probs, y_test.to_numpy(), top_n=top_n,asc=False)
display_model_output(test_probs, y_test.to_numpy(), top_n=top_n,asc=True)

In [ ]:
# model_final,sel_features_final,best_iteration = gen_top_mean_data_ml_final(test_combined_imp,cur_features,
#                         n_estimators = n_estimators,random_over_sampler=random_over_sampler,
#                         eval_metric=eval_metric,
#                         is_boruta_sel=is_boruta_sel,
#                          max_iter_boruta=max_iter_boruta)

linear

In [ ]:
print('Linear')
display_model_output(test_probs_linear, y_test.to_numpy(), top_n=top_n,asc=False)
print('Linear Final')
display_model_output(test_probs_linear_final, y_test.to_numpy(), top_n=top_n,asc=False)

In [ ]:
# actual_seed='5fb04950c9347ad2c512377d45270b9756fa29a8350260451d470bb254d87907'
# prev_seed='19a420a20c56cc633e077c2e9bb4f43c39ec8727e28e8a4ee4428f42a158a007'
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(prev_seed,
                                       test_all_k['client_seed'],
                                       nonce)
test_all_k['roll_actual_dummy']=roll_array

top_dummies = test_all_k.sort_values('roll_actual_dummy',ascending=False)[:1000]
display_model_output(top_dummies['roll_actual_dummy'].to_numpy(),
                     top_dummies['target'].to_numpy(), top_n=top_n,asc=False)
top_dummies[['roll_actual','roll_actual_dummy']].head(15)

In [ ]:
# test_all_k['roll_0_1_sum']=(test_all_k['roll_0']+test_all_k['roll_1'])
# top_data_temp = test_all_k.sort_values('probs_new',ascending=False)[:top_x]
# temp = top_data_temp.copy()
# # temp = top_data_temp.sort_values('roll_0_1_sum',ascending=False)
# display_model_output(temp['roll_0_1_sum'].to_numpy(), temp['target'].to_numpy(), top_n=top_n,asc=False)

In [ ]:
# top_data_temp[['roll_actual','roll_0_1_sum','probs_new', 'roll_0','roll_1']]\
#             .sort_values('roll_0_1_sum',ascending=False).iloc[3:4]
# #1768, 4451, 304, 4451,3601,3601,3601,3601,3601
# #50, 200, --, 500,1000,1500,2000

In [ ]:
# top_data_temp[['roll_actual','roll_0_1_sum','probs_new',
#                'roll_0','roll_1']].sort_values('probs_new',ascending=False)

In [ ]:
# top_data_temp[['roll_actual','roll_0_1_sum','probs_new']].sort_values('roll_0_1_sum',ascending=False)

In [ ]:
# for i in range(100):
#     test_sample = test_all_k.sample(n=20)
#     test_probs = (model_final.predict_proba(test_sample[cur_features])[:,1]) 
#     display_model_output(test_probs, test_sample['target'].to_numpy(), top_n=5,asc=top_order_asc)

In [ ]:

# for i in range(100):
#     test_sample = test_all_k.sample(n=6000)
#     test_probs= np.zeros(len(test_sample))
#     for model in cur_models:
#         test_probs += (model.predict_proba(test_sample[cur_features])[:,1]) 
#     test_probs = test_probs / len(cur_models)

#     display_model_output(test_probs, test_sample['target'].to_numpy(), top_n=5,asc=top_order_asc)

In [ ]:
test_all_k_imp[0].head()

In [ ]:
nonce

In [ ]:
# from xgboost import plot_tree
# import matplotlib

# plot_tree(cur_models[1], num_trees=0)
# fig = matplotlib.pyplot.gcf()
# fig.set_size_inches(150, 100)
# fig.savefig('tree.png')

In [ ]:
mask1= test_combined_imp['roll_mean_1_60']<4098
mask2= create_target_mask(test_combined_imp)

print(len(test_combined_imp[mask1]))
print(len(test_combined_imp[mask1 & mask2]))

mask1= test_all_k['roll_mean_1_60']<4098
mask2= create_target_mask(test_all_k)

print(len(test_all_k[mask1]))
print(len(test_all_k[mask1 & mask2])) 

#13% chance on both hash and actual on ['count_lt_1000_k_1_60']>2.5
#13.9% and 16% chance on hash and actual on ['roll_1'].astype('str').str.count('9') >=3

In [ ]:
mask1= test_combined_imp['roll_25'].astype('str').str.count('0') >=1
mask2= create_target_mask(test_combined_imp)

print(len(test_combined_imp[mask1]))
print(len(test_combined_imp[mask1 & mask2]))

mask1= test_all_k['roll_25'].astype('str').str.count('0') >=1
mask2= create_target_mask(test_all_k)

print(len(test_all_k[mask1]))
print(len(test_all_k[mask1 & mask2])) 

In [ ]:
key_cols = ['roll_actual','k','roll_0','roll_1','probs_new','probs','client_seed']
other_cols = [col for col in top_test.columns if col not in key_cols]
top_test_final[key_cols+other_cols]

END

In [ ]:
def create_cutoff_pos_df(test,models,features_test,test_hash_cutoff_df,
                         imp_feats_count_list,test_filt_all_k,
                         gen_probs=True,is_compute_matches=True,
                   test_ratio_limit=0.12,sim_cutoff_dfs=None,
                         low_high_record_condns=[(True,12,10),(False,10,10),(False,2,10)],
                        ratio_mean_asc=False,roll_mean_asc=True,
                        top_k_ele=None,is_print=False):

    cutoff_pos_df =pd.DataFrame(columns = ['imp_feats_count','condn_max','condn_val',
                                           'k_top_1','k_top_2','k_top_3','k_top_4','k_top_5',
                                          'mean_top_1', 'mean_top_2','mean_top_3','mean_top_4',
                                          'mean_top_5','match_pos_1',
                                          'match_total_top_5','match_total_top_10','match_ks_top_5'])

    
    max_good_k= test_hash_cutoff_df[test_hash_cutoff_df['ratio']<test_ratio_limit].index[0]-1
    print(f'{max_good_k=}')
    is_imp_feats = True 
    gen_sim_cutoff=False
    if sim_cutoff_dfs is None:
        gen_sim_cutoff=True
        sim_cutoff_dfs=[]
    elif gen_probs:
        test_probs = generate_probs(True,xgb_models,test,features_test) 
    else:
        test_probs = test
    top_mean_data_list=[]
    for i,imp_feats_count in enumerate(imp_feats_count_list):
        if is_print:
            print()
            print(f'********************** Feature count {imp_feats_count} *****************')
#         if gen_sim_cutoff:
#             sim_cutoff_df,test_probs= get_sim_cutoff_data(test,models,features_test,test_hash_cutoff_df,
#                                                      is_imp_feats,imp_feats_count,gen_probs)
#             sim_cutoff_dfs.append(sim_cutoff_df)
#         else:
#             sim_cutoff_df=sim_cutoff_dfs[i]
        
        for condn in low_high_record_condns:
#             print(f'******* Condition {condn} for feature count {imp_feats_count} *****************')
#             top_mean_data = get_hash_prob_top_data(test,models,features_test,
#                                        test_hash_cutoff_df,is_imp_feats,imp_feats_count,
#                                         test_filt_all_k,
#                                        sim_cutoff_df = None,
#                                         is_min_total=condn[0], total_cri=condn[1],
#                                         top_n=condn[2],
#                                         ratio_mean_asc=ratio_mean_asc,roll_mean_asc=roll_mean_asc)
            
            sim_summary= get_sim_summary(test_filt_all_k,test_hash_cutoff_df,
                                         is_imp_feats,imp_feats_count,is_print=is_print)
            top_mean_data = sim_summary.sort_values(['ratio_mean','roll_mean'],ascending=[ratio_mean_asc,roll_mean_asc])[:top_n].reset_index(drop=True)
            if is_print:
                print('--- Top 5 data ---')
                print(top_mean_data[:5])
            top_mean_data_list.append(top_mean_data)
            
            if top_k_ele is not None:
                print(f'\n******success data *****')
                mask= top_mean_data['k'].isin(top_k_ele)
                print(top_mean_data[mask])
                print(f'\n******success positions *****')
                cur_positions = list(top_mean_data[mask].index+1)
                print(cur_positions)
            
#             print(top_mean_data)
            top_5_mean_data = top_mean_data[:5]
            k_tops=list(top_5_mean_data['k'].values)
            mean_tops=list(top_5_mean_data['ratio_mean'].values)
            
            match_pos_1=np.nan
            match_total_top_5=np.nan
            match_total_top_10=np.nan
            match_ks_top_5=np.nan
            
            if is_compute_matches:
                
                top_mean_data['rank']=top_mean_data['ratio_mean'].rank(method='average',
                                                                  ascending=False)
                top_5_mean_data = top_mean_data[:5]
               
                matches_top_5=top_5_mean_data[top_5_mean_data['k']<=max_good_k]
                matches_top_5=matches_top_5[matches_top_5['ratio_mean']>0]
                match_total_top_5=len(matches_top_5)

                matches_top=top_mean_data[top_mean_data['k']<=max_good_k]
                matches_top=matches_top[matches_top['ratio_mean']>0]
                match_total_top_10 = len(matches_top)

                if match_total_top_5==0:
                    match_ks_top_5=[]
                    if match_total_top_10==0:
                        match_pos_1=11
                    else:
                        match_pos_1=matches_top.iloc[0]['rank']
                else:
    #                 match_pos_1=matches_top_5.index[0]+1
                    match_pos_1=matches_top_5.iloc[0]['rank']
                    match_ks_top_5 = list(matches_top_5['k'].values)
 
            
            result = [imp_feats_count,condn[0],condn[1]]+k_tops+mean_tops \
                    +[match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5]
        
#             print(cutoff_pos_df.columns)
#             print(f'{len(cutoff_pos_df.columns),len(result),result}')
            next_idx = len(cutoff_pos_df.index)
            cutoff_pos_df.loc[next_idx] = result
        
            if is_compute_matches:
                match_cols = [col for col in cutoff_pos_df if col.startswith('match')]
                print(cutoff_pos_df.loc[next_idx][match_cols])
        
    return cutoff_pos_df,sim_cutoff_dfs,top_mean_data_list


In [ ]:

def get_test_sel_cutoff_pos(test_probs,models,features_test,test_hash_cutoff_df,
                            cutoff_pos_df_hash,
                            is_min_total=False,total_cri=2,top_n=10,
                           sim_cutoff_dfs=None,imp_feats_count_list=None):
    cutoff_pos_stat = cutoff_pos_df_hash.groupby(['imp_feats_count'])['match_pos_1',
                                               'match_total_top_5',
                                               'match_total_top_10'].mean().reset_index()
    print(' ........... Overall Positions of Features .........')
    print(cutoff_pos_stat)
    cutoff_pos_df_hash_filt=cutoff_pos_df_hash[cutoff_pos_df_hash['condn_max']==False]
    cutoff_pos_stat = cutoff_pos_df_hash_filt.groupby(['imp_feats_count'])['match_pos_1',
                                               'match_total_top_5',
                                               'match_total_top_10'].mean().reset_index()
    print(' ........... Low Positions of Features .........')
    print(cutoff_pos_stat)
    cutoff_pos_best= cutoff_pos_stat.sort_values(['match_pos_1','match_total_top_5'],
                                                ascending=[True,False])\
                                    .reset_index().iloc[0]
    print('*** Best cutoff position in hash **** ')
    print(cutoff_pos_best)
    predicted_raw_pos=cutoff_pos_best['match_pos_1']
    predicted_max_pos = math.ceil(predicted_raw_pos)
    
    if predicted_raw_pos==predicted_max_pos:
        predicted_max_pos+=1
    
    is_imp_feats=True
    imp_feats_count=cutoff_pos_best['imp_feats_count']
    is_min_total=False
    
    if sim_cutoff_dfs is None:
        sim_cutoff_df,test_probs= get_sim_cutoff_data(test_probs,models,features_test,test_hash_cutoff_df,
                                                 is_imp_feats,imp_feats_count,gen_probs=False)
    else:
        #to use existing buffer, read the matching sim cutoff df from the list of buffer
        idx = imp_feats_count_list.index(imp_feats_count)
        sim_cutoff_df=sim_cutoff_dfs[idx]

    top_mean_data = get_hash_prob_top_data(test_probs,models,features_test,
                           test_hash_cutoff_df,is_imp_feats,imp_feats_count,
                           sim_cutoff_df = sim_cutoff_df,
                            is_min_total=is_min_total, total_cri=total_cri,
                            top_n=top_n)
    
    top_mean_data['rank']=top_mean_data['ratio_mean'].rank(method='min',ascending=False)
    mask = (top_mean_data['rank']<=predicted_max_pos) & (top_mean_data['ratio_mean']>0)
    top_mean_sel = top_mean_data[mask]
    k_sel_list = list(top_mean_sel['k'].values)
    print(top_mean_data)
    print(top_mean_sel)
    print(f'{k_sel_list=}')
    

#     mask = cutoff_pos_df_actual['imp_feats_count']==cutoff_pos_best['cutoff_pos_best']
#     mask2 = cutoff_pos_df_actual['match_pos_1']<=predicted_max_pos
#     k_sel = cutoff_pos_df_actual.loc[mask & mask2, 'k' ]
    
    test_sel  = None
    for k_sel in k_sel_list:
        row = test_hash_cutoff_df.loc[k_sel]
    #     print(row)
        mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
        cur_test_sel = test_probs[mask]
        cur_test_sel['k']=k_sel
        if test_sel is None:
            test_sel = cur_test_sel
        else:
            test_sel = pd.concat([test_sel,cur_test_sel],axis=0)
        print('Test Size:',len(test_sel))

    return k_sel_list,test_sel,top_mean_sel



In [ ]:
opt_count = 4

HASH_IDX = 0
RATIO_MEAN_ASC_IDX = 1
ROLL_MEAN_ASC_IDX = 2
K_IDX = 3
IMP_FEATS_IDX = 4

In [ ]:
def create_k_order(end,top_n,hash_idx,ratio_mean_order,roll_mean_order,
                  all_ks,imp_feats_count_list_norm_idx,imp_feats_count_list_norm_arr,
                   top_mean_actual,k_order_arr):
    start=end
    end = start + top_n
#     print(f'start {start} to {end}')

    k_order_arr[HASH_IDX,start:end] = hash_idx
    k_order_arr[RATIO_MEAN_ASC_IDX,start:end] = ratio_mean_order
    k_order_arr[ROLL_MEAN_ASC_IDX,start:end] = roll_mean_order
    k_order_arr[K_IDX,start:end] = np.sort(all_ks)

    #for each imp feats count, add only one set of k values and their corresponding positions
    # in that imp feats count
    for i,sel_count_idx in enumerate(imp_feats_count_list_norm_idx):
#         print('cur feat:',imp_feats_count_list_norm_arr[i])
        top_mean_filt = top_mean_actual[sel_count_idx].sort_values('k')
        k_order_arr[IMP_FEATS_IDX+i,start:end] = list(top_mean_filt.index+1)
        
    return k_order_arr,end

In [ ]:
# imp_feats_count_list = [2,5,10,15,20,25,30,35,40,45,50,55,60]
# imp_feats_count_list= [2]+list(range(5,201,5))
# imp_feats_count_list= list(range(2,151))
imp_feats_count_list= list(range(2,151))
# imp_feats_count_list_norm_arr = np.array([2]+list(range(5,61,5)))
imp_feats_count_list_norm_arr = np.array(imp_feats_count_list)

# imp_feats_count_list= list(range(2,11))
# imp_feats_count_list_norm_arr = np.array([2]+list(range(5,11,5)))

imp_feats_count_list_norm_idx = np.where(np.isin(imp_feats_count_list,
                                                 imp_feats_count_list_norm_arr))[0]
# imp_feats_count_list_norm_idx = imp_feats_count_list_norm_arr-2



imp_feats_size_norm = len(imp_feats_count_list_norm_idx)
# imp_feats_count_list = [2,5]
imp_feats_size = len(imp_feats_count_list)

###### Normal Test Hash Data Based Match Pos df generation

Check top important hash test data for ascending or descending ratio mean performance

In [ ]:
%%time
first_match_pos_arr= np.zeros((hash_imp_count,2,imp_feats_size))
dictionary_list=[]

total_k_all_hash=0
for hash_idx in range(hash_imp_count):
    total_k_all_hash += len(all_k_imp[hash_idx])
    
k_order_arr = np.zeros((4+imp_feats_size_norm,
                        opt_count*total_k_all_hash))


end =0

for hash_idx in range(hash_imp_count):
    k_count = len(all_k_imp[hash_idx])
    top_n = k_count
    
    print(f'\ntop_n:{top_n}')
    for ratio_mean_order in [False,True]:
        for roll_mean_order in [False,True]:
            print(f'********************** Test Hash idx:{hash_idx} {hash_imp_names[hash_idx]} (ratio_mean order) {ratio_mean_order} (roll_mean order) {roll_mean_order} ***********************')
            sim_cutoff_dfs_actual=None
            cutoff_pos_df_actual,sim_cutoff_dfs_actual,top_mean_actual=create_cutoff_pos_df(test_hash_imp[hash_idx],xgb_models,
                                features_test,df[df.index.isin(all_k_imp[hash_idx])],
                                imp_feats_count_list,test_all_k_imp[hash_idx],
                                is_compute_matches=False,
                               test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
                                low_high_record_condns=[(False,1,top_n)],
                                ratio_mean_asc=ratio_mean_order,
                                roll_mean_asc=roll_mean_order,
                                is_print=False,     
                                )
            
            k_order_arr,end = create_k_order(end,top_n,hash_idx,ratio_mean_order,roll_mean_order,
                  all_k_imp[hash_idx],imp_feats_count_list_norm_idx,imp_feats_count_list_norm_arr,
                   top_mean_actual,k_order_arr)
            
            first_match_pos_list=[]
            match_pos_list=[]
            for feats_idx in range(imp_feats_size):
                cur_top_data = top_mean_actual[feats_idx]
#                 print(f'\n******{imp_feats_count_list[feats_idx]} feats success data *****')
                mask= cur_top_data['k'].isin(top_k_ele_imp[hash_idx])
#                 print(cur_top_data[mask]['k'].to_numpy())
                cur_positions = list(cur_top_data[mask].index+1)

                dictionary_data = {'imp_feats_count': imp_feats_count_list[feats_idx],
                                   'hash_index':hash_idx, 
                                   'hash_feat_name': hash_imp_names[hash_idx], 
                                    'ratio_mean_asc':ratio_mean_order,
                                    'roll_mean_asc':roll_mean_order,
                                    'first_match_pos':cur_positions[0],
                                    'other_match_pos':cur_positions[1:]
                      }
                dictionary_list.append(dictionary_data)
#                 print(f'\n {imp_feats_count_list[feats_idx]} feats success positions')
#                 print(cur_positions)
        
        
match_pos_df = pd.DataFrame.from_dict(dictionary_list)

imp_feats_count_cols  = [f'imp_feats_{val}' for val in imp_feats_count_list_norm_arr]

k_order_df = pd.DataFrame(k_order_arr.T,columns=['hash_idx','ratio_mean_asc','roll_mean_asc',
                                              'k']+imp_feats_count_cols)
for col in k_order_df.columns:
    k_order_df[col]= k_order_df[col].astype('int')
k_order_df.head()
# match_pos_df        

In [ ]:
match_pos_df.to_csv(f'data/analysis/match_pos_df_hash_{nonce}_{k_bin_size}.csv',index=False)
k_order_df.to_csv(f'data/analysis/k_order_df_hash_{nonce}_{k_bin_size}.csv',index=False)

In [ ]:
# imp_feats_count_list= [2]+list(range(2,151))

In [ ]:
%%time
# imp_feats_count_list = [2,5,10,15,20,25,30,35,40,45,50,55,60]
# imp_feats_count_list= [2]+list(range(5,201,5))
# imp_feats_count_list= list(range(2,151))
# imp_feats_count_list= list(range(150,250,10))

# imp_feats_count_list = [2,5]
imp_feats_size = len(imp_feats_count_list)
dictionary_list=[]

top_n = len(all_k)
print(f'\ntop_n:{top_n}')
k_order_arr_actual = np.zeros((4+imp_feats_size_norm,
                        opt_count*top_n))
end = 0
hash_idx =-1
for ratio_mean_order in [False,True]:
    for roll_mean_order in [False,True]:

        print(f'****************** ACTUAL ratio_mean asc {ratio_mean_order} roll_mean asc {roll_mean_order} *******************')
        sim_cutoff_dfs_actual=None
        cutoff_pos_df_actual,sim_cutoff_dfs_actual,top_mean_actual=create_cutoff_pos_df(test,xgb_models,
                            features_test,df[df.index.isin(all_k)],
                            imp_feats_count_list,test_all_k,is_compute_matches=False,
                           test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
                            low_high_record_condns=[(False,1,top_n)],
                            ratio_mean_asc=ratio_mean_order,
                            roll_mean_asc=roll_mean_order,#not(HIGH_ANALYSIS)
                            )
        k_order_arr_actual,end = create_k_order(end,top_n,hash_idx,ratio_mean_order,roll_mean_order,
                          all_k,imp_feats_count_list_norm_idx,imp_feats_count_list_norm_arr,
                           top_mean_actual,k_order_arr_actual)

        for feats_idx in range(imp_feats_size):
            cur_top_data = top_mean_actual[feats_idx]
#             print(f'\n******{imp_feats_count_list[feats_idx]} feats success data *****')
            mask= cur_top_data['k'].isin(top_k_ele)
#             print(cur_top_data[mask])
            cur_positions = list(cur_top_data[mask].index+1)

            dictionary_data = {'imp_feats_count': imp_feats_count_list[feats_idx],
                               'hash_index':hash_idx, 
                               'hash_feat_name': 'ACTUAL', 
                                'ratio_mean_asc':ratio_mean_order,
                                'roll_mean_asc':roll_mean_order,
                                'first_match_pos':cur_positions[0],
                                'other_match_pos':cur_positions[1:]
                  }
            dictionary_list.append(dictionary_data)
#             print(f'\n {imp_feats_count_list[feats_idx]} feats success positions')
#             print(cur_positions)


match_pos_df_actual = pd.DataFrame.from_dict(dictionary_list)

print()
print('First Match Pos Mean:',match_pos_df_actual.groupby(['ratio_mean_asc','roll_mean_asc'])['first_match_pos'].mean().reset_index())
print()
# match_pos_df_actual.head()  

imp_feats_count_cols  = [f'imp_feats_{val}' for val in imp_feats_count_list_norm_arr]

k_order_df_actual = pd.DataFrame(k_order_arr_actual.T,
                                 columns=['hash_idx','ratio_mean_asc','roll_mean_asc',
                                          'k']+imp_feats_count_cols)
for col in k_order_df.columns:
    k_order_df_actual[col]= k_order_df_actual[col].astype('int')
k_order_df_actual.head()


In [ ]:
match_pos_df_actual.to_csv(f'data/analysis/match_pos_df_actual_{nonce}_{k_bin_size}.csv',index=False)
k_order_df_actual.to_csv(f'data/analysis/k_order_df_actual_{nonce}_{k_bin_size}.csv',index=False)

In [ ]:
sel_feats = [2]+list(range(5,61,5))
# sel_feats = None

if isinstance(sel_feats, list):
    print('sel_feats:',sel_feats)
    mask = match_pos_df['imp_feats_count'].isin(sel_feats)
    match_pos_filt_feats = match_pos_df[mask]
else:
    match_pos_filt_feats = match_pos_df
# mask_hash_count = match_pos_filt_feats['hash_index']<=2
# match_pos_filt_feats=   match_pos_filt_feats[mask_hash_count]
match_pos_filt_feats.head()

In [ ]:
        
match_summary =match_pos_filt_feats.groupby(['hash_index','ratio_mean_asc','roll_mean_asc'])\
                            .agg(first_match_pos_mean =('first_match_pos','mean'))
                                                                        
print(match_summary.reset_index())
match_top_pos = match_summary['first_match_pos_mean'].groupby('hash_index', group_keys=False).nsmallest(1)
match_top_pos = match_top_pos.reset_index()
print(match_top_pos)


match_freq = match_top_pos.groupby(['ratio_mean_asc','roll_mean_asc']).size().sort_values(ascending=False)
print(match_freq.reset_index())
top_ratio_mean_asc = match_freq.index[0]  
# if len(match_value_counts)==2:
#     ratio_mean_order_equal = match_top_pos['ratio_mean_asc'].value_counts().iloc[0]==match_top_pos['ratio_mean_asc'].value_counts().iloc[1]
#     if ratio_mean_order_equal:
#         print('PROJECT WARNING: Both Ratio mean order ascending and descending are equal')
print(f'\nTop options by frequency : Ratio mean asc={top_ratio_mean_asc[0]}  Roll mean asc={top_ratio_mean_asc[1]}')

In [ ]:
match_mean = match_pos_filt_feats.groupby(['ratio_mean_asc','roll_mean_asc'])['first_match_pos'].mean().sort_values(ascending=True)
print(match_mean.reset_index())
top_options = match_mean.index[0]  
print(f'\nTop options by mean : Ratio mean asc={top_options[0]}  Roll mean asc={top_options[1]}')

In [ ]:
# sel_ratio_mean_asc = top_options[0]
# sel_roll_mean_asc = top_options[1]

sel_ratio_mean_asc = True   
sel_roll_mean_asc =  False

In [ ]:
# match_pos_df.groupby('hash_index')
# match_pos_df['rank'] =  match_pos_df['first_match_pos'].rank(method='average',
#                                                              ascending=True)
# mask = (match_pos_df['ratio_mean_asc']==top_ratio_mean_asc)
# pos_top_df= match_pos_df[mask].sort_values(['hash_index','first_match_pos'],ascending=True)\
#             .groupby(['hash_index'])\
#             .head(5)
# print(pos_top_df['imp_feats_count'].value_counts())
# pos_top_df

mask = (match_pos_filt_feats['ratio_mean_asc']==sel_ratio_mean_asc) \
        & (match_pos_filt_feats['roll_mean_asc']==sel_roll_mean_asc)
match_pos_df_filt=match_pos_filt_feats[mask]
match_pos_df_filt['rank'] = match_pos_df_filt.groupby('hash_index')['first_match_pos'].rank("min")
mask2=match_pos_df_filt['rank']<=5
match_pos_df_filt=match_pos_df_filt[mask2]
print(match_pos_df_filt['imp_feats_count'].value_counts())
match_pos_df_filt
# match_pos_df.sort_values(['first_match_pos'],ascending=True)\
#             .groupby(['hash_index','ratio_mean_asc'])\
#             .head(5)\
#             .sort_values(['hash_index','ratio_mean_asc'],ascending=True)
    

In [ ]:
# match_pos_df.groupby('hash_index')
# match_pos_df['rank'] =  match_pos_df['first_match_pos'].rank(method='average',
#                                                              ascending=True)
mask = (match_pos_filt_feats['ratio_mean_asc']==sel_ratio_mean_asc) \
        & (match_pos_filt_feats['roll_mean_asc']==sel_roll_mean_asc)
# mask = (match_pos_df['ratio_mean_asc']==False)
pos_top_df= match_pos_filt_feats[mask].sort_values(['hash_index','first_match_pos'],ascending=True)\
            .groupby(['imp_feats_count'])\
            .agg(mean =('first_match_pos','mean'),\
                 std =('first_match_pos','std'),
                median = ('first_match_pos','median'),
                minimum = ('first_match_pos','min'),
                maximum = ('first_match_pos','max'),)
#             .sort_values('imp_feats_count')
# print(pos_top_df['imp_feats_count'].value_counts())
pos_mean_summary = pos_top_df.sort_values(['mean','minimum','maximum'],ascending=[True,True,True]).reset_index()
pos_mean_summary
# match_pos_df.sort_values(['first_match_pos'],ascending=True)\
#             .groupby(['hash_index','ratio_mean_asc'])\
#             .head(5)\
#             .sort_values(['hash_index','ratio_mean_asc'],ascending=True)
    

In [ ]:
sel_feats_count = pos_mean_summary['imp_feats_count'].iloc[0]
sel_feats_count

In [ ]:
# pos_top_imp_feats = list(pos_top_df['imp_feats_count'].value_counts().index)
# imp_feats_count_arr = np.array(imp_feats_count_list)
# # itemindex = np.where(imp_feats_count_arr in pos_top_imp_feats)
# print(pos_top_imp_feats)
# itemindex= np.searchsorted(imp_feats_count_arr, pos_top_imp_feats)
# itemindex

In [ ]:
if isinstance(sel_feats, list):
    print('sel_feats:',sel_feats)
    mask = match_pos_df_actual['imp_feats_count'].isin(sel_feats)
    match_pos_actual_filt_feats = match_pos_df_actual[mask]
else:
    match_pos_actual_filt_feats = match_pos_df_actual

# match_pos_actual_filt_feats.head()

In [ ]:
mask = (match_pos_actual_filt_feats['ratio_mean_asc']==sel_ratio_mean_asc) \
        & (match_pos_actual_filt_feats['roll_mean_asc']==sel_roll_mean_asc)
match_pos_actual_filt=match_pos_actual_filt_feats[mask]
match_pos_actual_filt.sort_values('first_match_pos',ascending=True)

In [ ]:
match_pos_actual_filt

In [ ]:
len(match_pos_df_actual)
print('First Match Pos Mean:',match_pos_actual_filt_feats.groupby(['ratio_mean_asc','roll_mean_asc'])['first_match_pos'].mean().reset_index())

In [ ]:
%%time
# imp_feats_count_list = [2]
imp_feats_count_list = [sel_feats_count]

# imp_feats_count_list= [2]+list(range(5,201,5))
# imp_feats_count_list= list(range(2,151))

sim_cutoff_dfs_actual=None
top_n = len(all_k)
cutoff_pos_df_actual,sim_cutoff_dfs_actual,top_mean_actual=create_cutoff_pos_df(test,xgb_models,
                    features_test,df[df.index.isin(all_k)],
                    imp_feats_count_list,test_all_k,is_compute_matches=False,
                   test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
                    low_high_record_condns=[(False,1,top_n)],
                    ratio_mean_asc=sel_ratio_mean_asc,
                    roll_mean_asc=sel_roll_mean_asc,
                    top_k_ele=top_k_ele,is_print=False)
cutoff_pos_df_actual

In [ ]:
%%time
k_list = top_mean_actual[0][:5]['k'].values
test_sel=test_all_k[test_all_k['k'].isin(k_list)]
print(k_list)
test_sel[['k','roll_actual','client_seed','roll_0','roll_1']]

In [ ]:
top_k_ele_imp

In [ ]:
col_to_sort = f'imp_feats_{sel_feats_count}'
cols = ['hash_idx','k',col_to_sort]
mask_temp = (k_order_df_actual['ratio_mean_asc']==sel_ratio_mean_asc) & (k_order_df_actual['roll_mean_asc']==sel_roll_mean_asc) 
k_order_df_filt_actual = k_order_df_actual[mask_temp]
k_order_df_filt_actual = k_order_df_filt_actual.sort_values(col_to_sort)
k_list = k_order_df_filt_actual['k'][:5].to_numpy()
print(k_list)
k_order_df_filt_actual[cols].head(5)

In [ ]:
# # top_k_ele_imp
# top_k_ele_all=[val for sublist in top_k_ele_imp for val in sublist]
# all_k_ele_imp=[val for sublist in all_k_imp for val in sublist]
# # sorted(x)

# # diff = set(all_k).difference(set(top_k_ele_all))
# diff = set(all_k).difference(set(all_k_ele_imp))
# print(len(all_k),len(diff))
# print(diff)

In [ ]:
cols = ['hash_idx','k',col_to_sort,'ratio_mean_asc','roll_mean_asc']
mask_temp = (k_order_df['ratio_mean_asc']==sel_ratio_mean_asc) & (k_order_df['roll_mean_asc']==sel_roll_mean_asc) \
            & (k_order_df['k'].isin(k_list))
# mask_temp = (k_order_df['k'].isin(k_list))
k_order_df_filt = k_order_df[mask_temp]
imp_feats_cols= [col for col in k_order_df_filt.columns if col.startswith('imp_feats')]
k_order_df_filt['imp_mean']=k_order_df_filt[imp_feats_cols].mean(axis=1)

k_order_df_filt = k_order_df_filt.sort_values(col_to_sort)
print(k_order_df_filt['k'].unique())
k_order_df_filt[['imp_mean']+cols]


In [ ]:
#prev seed matches in k order
cols = ['hash_idx','k',col_to_sort,'ratio_mean_asc','roll_mean_asc']
mask_temp = (k_order_df['k'].isin( top_k_ele )) # prev seed success

k_order_df_filt = k_order_df[mask_temp]
imp_feats_cols= [col for col in k_order_df_filt.columns if col.startswith('imp_feats')]
k_order_df_filt['imp_mean']=k_order_df_filt[imp_feats_cols].mean(axis=1)

k_order_df_filt = k_order_df_filt.sort_values('imp_mean')
print(k_order_df_filt['k'].unique())
k_order_df_filt[['imp_mean']+cols]


###### Test Hash Imp Based Match Pos Generation

In [ ]:
# test_hash = test_hash_imp[0].copy()

In [ ]:
# %%time

# k_bin_size=700

# print(f'***************** Bin {k_bin_size} ***************** ')
# df= gen_hash_cutoff_df(xgb_models,features_test,test_hash,
#                             k_bin_size=k_bin_size,quant=0.95)
# print('Test Hash Cutoff Df Stats')
# print('Total size:',len(df[df['total']==1]))
# print('Success size:',len(df[(df['total']==1) & (df['ratio']>0)]))

# all_k,top_k,test_all_k = gen_all_k(xgb_models,features_test,
#                                 test,df,target_total=1)

# top_k_ele = [x[0] for x in top_k]
# print(top_k_ele)
# print('Test Size:',len(test_all_k))
# test_all_k.head()

In [ ]:
# df[df['total']==1]

In [ ]:
# key_cols = ['k','roll_actual','probs','roll_0','roll_1','client_seed']
# other_cols = [col for col in test_all_k if col not in key_cols]
# test_all_k[key_cols+other_cols]

In [ ]:
# # key_cols = ['k','roll_actual','client_seed','probs','roll_0','roll_1']
# other_cols = [col for col in test_all_k if col not in key_cols]
# test_all_k[test_all_k['k'].isin(top_k_ele)][key_cols+other_cols]

Important Feats Top K Elements

In [ ]:
# %%time

# # k_bin_size=600

# all_k_imp =[-1]*hash_imp_count
# top_k_imp =[-1]*hash_imp_count
# test_all_k_imp =[-1]*hash_imp_count
# top_k_ele_imp =[-1]*hash_imp_count

# df= gen_hash_cutoff_df(xgb_models,features_test,test_hash,
#                             k_bin_size=k_bin_size,quant=0.95)
# print('Test Hash Cutoff Df Stats')
# print('Total size:',len(df[df['total']==1]))
# print('Success size:',len(df[(df['total']==1) & (df['ratio']>0)]))

# for hash_idx in range(hash_imp_count):

#     print(f'***************** Test Hash {hash_imp_names[hash_idx]} ***************** ')

#     all_k_imp[hash_idx],top_k_imp[hash_idx],test_all_k_imp[hash_idx] = gen_all_k(xgb_models,features_test,
#                                                                         test_hash_imp[hash_idx],
#                                                                         df,target_total=1)

#     top_k_ele_imp[hash_idx] = [x[0] for x in top_k_imp[hash_idx]]


Check top important hash test data for ascending or descending ratio mean performance

In [ ]:
# %%time
# first_match_pos_arr= np.zeros((hash_imp_count,2,imp_feats_size))
# dictionary_list=[]

# total_k_all_hash=0
# for hash_idx in range(hash_imp_count):
#     total_k_all_hash += len(all_k_imp[hash_idx])
    
# k_order_arr = np.zeros((4+imp_feats_size_norm,
#                         opt_count*total_k_all_hash))


# end =0

# for hash_idx in range(hash_imp_count):
#     k_count = len(all_k_imp[hash_idx])
#     top_n = k_count
    
#     print(f'\ntop_n:{top_n}')
#     for ratio_mean_order in [False,True]:
#         for roll_mean_order in [False,True]:
#             print(f'********************** Test Hash idx:{hash_idx} {hash_imp_names[hash_idx]} (ratio_mean order) {ratio_mean_order} (roll_mean order) {roll_mean_order} ***********************')
#             sim_cutoff_dfs_actual=None
#             cutoff_pos_df_actual,sim_cutoff_dfs_actual,top_mean_actual=create_cutoff_pos_df(test_hash_imp[hash_idx],xgb_models,
#                                 features_test,df[df.index.isin(all_k_imp[hash_idx])],
#                                 imp_feats_count_list,test_all_k_imp[hash_idx],
#                                 is_compute_matches=False,
#                                test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
#                                 low_high_record_condns=[(False,1,top_n)],
#                                 ratio_mean_asc=ratio_mean_order,
#                                 roll_mean_asc=roll_mean_order,
#                                 is_print=False,     
#                                 )
            
#             k_order_arr,end = create_k_order(end,top_n,hash_idx,ratio_mean_order,roll_mean_order,
#                   all_k_imp[hash_idx],imp_feats_count_list_norm_idx,imp_feats_count_list_norm_arr,
#                    top_mean_actual,k_order_arr)
            
#             first_match_pos_list=[]
#             match_pos_list=[]
#             for feats_idx in range(imp_feats_size):
#                 cur_top_data = top_mean_actual[feats_idx]
# #                 print(f'\n******{imp_feats_count_list[feats_idx]} feats success data *****')
#                 mask= cur_top_data['k'].isin(top_k_ele_imp[hash_idx])
# #                 print(cur_top_data[mask]['k'].to_numpy())
#                 cur_positions = list(cur_top_data[mask].index+1)

#                 dictionary_data = {'imp_feats_count': imp_feats_count_list[feats_idx],
#                                    'hash_index':hash_idx, 
#                                    'hash_feat_name': hash_imp_names[hash_idx], 
#                                     'ratio_mean_asc':ratio_mean_order,
#                                     'roll_mean_asc':roll_mean_order,
#                                     'first_match_pos':cur_positions[0],
#                                     'other_match_pos':cur_positions[1:]
#                       }
#                 dictionary_list.append(dictionary_data)
# #                 print(f'\n {imp_feats_count_list[feats_idx]} feats success positions')
# #                 print(cur_positions)
        
        
# match_pos_df = pd.DataFrame.from_dict(dictionary_list)

# imp_feats_count_cols  = [f'imp_feats_{val}' for val in imp_feats_count_list_norm_arr]

# k_order_df = pd.DataFrame(k_order_arr.T,columns=['hash_idx','ratio_mean_asc','roll_mean_asc',
#                                               'k']+imp_feats_count_cols)
# for col in k_order_df.columns:
#     k_order_df[col]= k_order_df[col].astype('int')
# k_order_df.head()
# # match_pos_df        

In [ ]:
# match_pos_df.to_csv(f'data/analysis/match_pos_df_hash_{nonce}_{k_bin_size}_hashimp.csv',index=False)
# k_order_df.to_csv(f'data/analysis/k_order_df_hash_{nonce}_{k_bin_size}_hashimp.csv',index=False)

In [ ]:
# imp_feats_count_list= [2]+list(range(2,151))

In [ ]:
# %%time
# # imp_feats_count_list = [2,5,10,15,20,25,30,35,40,45,50,55,60]
# # imp_feats_count_list= [2]+list(range(5,201,5))
# # imp_feats_count_list= list(range(2,151))
# # imp_feats_count_list= list(range(150,250,10))

# # imp_feats_count_list = [2,5]
# imp_feats_size = len(imp_feats_count_list)
# dictionary_list=[]

# top_n = len(all_k)
# print(f'\ntop_n:{top_n}')
# k_order_arr_actual = np.zeros((4+imp_feats_size_norm,
#                         opt_count*top_n))
# end = 0
# hash_idx =-1
# for ratio_mean_order in [False,True]:
#     for roll_mean_order in [False,True]:

#         print(f'****************** ACTUAL ratio_mean asc {ratio_mean_order} roll_mean asc {roll_mean_order} *******************')
#         sim_cutoff_dfs_actual=None
#         cutoff_pos_df_actual,sim_cutoff_dfs_actual,top_mean_actual=create_cutoff_pos_df(test,xgb_models,
#                             features_test,df[df.index.isin(all_k)],
#                             imp_feats_count_list,test_all_k,is_compute_matches=False,
#                            test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
#                             low_high_record_condns=[(False,1,top_n)],
#                             ratio_mean_asc=ratio_mean_order,
#                             roll_mean_asc=roll_mean_order,#not(HIGH_ANALYSIS)
#                             )
#         k_order_arr_actual,end = create_k_order(end,top_n,hash_idx,ratio_mean_order,roll_mean_order,
#                           all_k,imp_feats_count_list_norm_idx,imp_feats_count_list_norm_arr,
#                            top_mean_actual,k_order_arr_actual)

#         for feats_idx in range(imp_feats_size):
#             cur_top_data = top_mean_actual[feats_idx]
# #             print(f'\n******{imp_feats_count_list[feats_idx]} feats success data *****')
#             mask= cur_top_data['k'].isin(top_k_ele)
# #             print(cur_top_data[mask])
#             cur_positions = list(cur_top_data[mask].index+1)

#             dictionary_data = {'imp_feats_count': imp_feats_count_list[feats_idx],
#                                'hash_index':hash_idx, 
#                                'hash_feat_name': 'ACTUAL', 
#                                 'ratio_mean_asc':ratio_mean_order,
#                                 'roll_mean_asc':roll_mean_order,
#                                 'first_match_pos':cur_positions[0],
#                                 'other_match_pos':cur_positions[1:]
#                   }
#             dictionary_list.append(dictionary_data)
# #             print(f'\n {imp_feats_count_list[feats_idx]} feats success positions')
# #             print(cur_positions)


# match_pos_df_actual = pd.DataFrame.from_dict(dictionary_list)

# print()
# print('First Match Pos Mean:',match_pos_df_actual.groupby(['ratio_mean_asc','roll_mean_asc'])['first_match_pos'].mean().reset_index())
# print()
# # match_pos_df_actual.head()  

# imp_feats_count_cols  = [f'imp_feats_{val}' for val in imp_feats_count_list_norm_arr]

# k_order_df_actual = pd.DataFrame(k_order_arr_actual.T,
#                                  columns=['hash_idx','ratio_mean_asc','roll_mean_asc',
#                                           'k']+imp_feats_count_cols)
# for col in k_order_df.columns:
#     k_order_df_actual[col]= k_order_df_actual[col].astype('int')
# k_order_df_actual.head()


In [ ]:
# match_pos_df_actual.to_csv(f'data/analysis/match_pos_df_actual_{nonce}_{k_bin_size}_hashimp.csv',index=False)
# k_order_df_actual.to_csv(f'data/analysis/k_order_df_actual_{nonce}_{k_bin_size}_hashimp.csv',index=False)

In [ ]:
# sel_feats = [2]+list(range(5,61,5))
# # sel_feats = None

# if isinstance(sel_feats, list):
#     print('sel_feats:',sel_feats)
#     mask = match_pos_df['imp_feats_count'].isin(sel_feats)
#     match_pos_filt_feats = match_pos_df[mask]
# else:
#     match_pos_filt_feats = match_pos_df
# # mask_hash_count = match_pos_filt_feats['hash_index']<=2
# # match_pos_filt_feats=   match_pos_filt_feats[mask_hash_count]
# match_pos_filt_feats.head()

In [ ]:
        
# match_summary =match_pos_filt_feats.groupby(['hash_index','ratio_mean_asc','roll_mean_asc'])\
#                             .agg(first_match_pos_mean =('first_match_pos','mean'))
                                                                        
# print(match_summary.reset_index())
# match_top_pos = match_summary['first_match_pos_mean'].groupby('hash_index', group_keys=False).nsmallest(1)
# match_top_pos = match_top_pos.reset_index()
# print(match_top_pos)


# match_freq = match_top_pos.groupby(['ratio_mean_asc','roll_mean_asc']).size().sort_values(ascending=False)
# print(match_freq.reset_index())
# top_ratio_mean_asc = match_freq.index[0]  
# # if len(match_value_counts)==2:
# #     ratio_mean_order_equal = match_top_pos['ratio_mean_asc'].value_counts().iloc[0]==match_top_pos['ratio_mean_asc'].value_counts().iloc[1]
# #     if ratio_mean_order_equal:
# #         print('PROJECT WARNING: Both Ratio mean order ascending and descending are equal')
# print(f'\nTop options by frequency : Ratio mean asc={top_ratio_mean_asc[0]}  Roll mean asc={top_ratio_mean_asc[1]}')

In [ ]:
# match_mean = match_pos_filt_feats.groupby(['ratio_mean_asc','roll_mean_asc'])['first_match_pos'].mean().sort_values(ascending=True)
# print(match_mean.reset_index())
# top_options = match_mean.index[0]  
# print(f'\nTop options by mean : Ratio mean asc={top_options[0]}  Roll mean asc={top_options[1]}')

In [ ]:
# # sel_ratio_mean_asc = top_options[0]
# # sel_roll_mean_asc = top_options[1]

# sel_ratio_mean_asc =  False 
# sel_roll_mean_asc = False

In [ ]:
# # match_pos_df.groupby('hash_index')
# # match_pos_df['rank'] =  match_pos_df['first_match_pos'].rank(method='average',
# #                                                              ascending=True)
# # mask = (match_pos_df['ratio_mean_asc']==top_ratio_mean_asc)
# # pos_top_df= match_pos_df[mask].sort_values(['hash_index','first_match_pos'],ascending=True)\
# #             .groupby(['hash_index'])\
# #             .head(5)
# # print(pos_top_df['imp_feats_count'].value_counts())
# # pos_top_df

# mask = (match_pos_filt_feats['ratio_mean_asc']==sel_ratio_mean_asc) \
#         & (match_pos_filt_feats['roll_mean_asc']==sel_roll_mean_asc)
# match_pos_df_filt=match_pos_filt_feats[mask]
# match_pos_df_filt['rank'] = match_pos_df_filt.groupby('hash_index')['first_match_pos'].rank("min")
# mask2=match_pos_df_filt['rank']<=5
# match_pos_df_filt=match_pos_df_filt[mask2]
# print(match_pos_df_filt['imp_feats_count'].value_counts())
# match_pos_df_filt
# # match_pos_df.sort_values(['first_match_pos'],ascending=True)\
# #             .groupby(['hash_index','ratio_mean_asc'])\
# #             .head(5)\
# #             .sort_values(['hash_index','ratio_mean_asc'],ascending=True)
    

In [ ]:
# # match_pos_df.groupby('hash_index')
# # match_pos_df['rank'] =  match_pos_df['first_match_pos'].rank(method='average',
# #                                                              ascending=True)
# mask = (match_pos_filt_feats['ratio_mean_asc']==sel_ratio_mean_asc) \
#         & (match_pos_filt_feats['roll_mean_asc']==sel_roll_mean_asc)
# # mask = (match_pos_df['ratio_mean_asc']==False)
# pos_top_df= match_pos_filt_feats[mask].sort_values(['hash_index','first_match_pos'],ascending=True)\
#             .groupby(['imp_feats_count'])\
#             .agg(mean =('first_match_pos','mean'),\
#                  std =('first_match_pos','std'),
#                 median = ('first_match_pos','median'),
#                 minimum = ('first_match_pos','min'),
#                 maximum = ('first_match_pos','max'),)
# #             .sort_values('imp_feats_count')
# # print(pos_top_df['imp_feats_count'].value_counts())
# pos_mean_summary = pos_top_df.sort_values(['mean','minimum','maximum'],ascending=[True,True,True]).reset_index()
# pos_mean_summary
# # match_pos_df.sort_values(['first_match_pos'],ascending=True)\
# #             .groupby(['hash_index','ratio_mean_asc'])\
# #             .head(5)\
# #             .sort_values(['hash_index','ratio_mean_asc'],ascending=True)
    

In [ ]:
# pos_top_imp_feats = list(pos_top_df['imp_feats_count'].value_counts().index)
# imp_feats_count_arr = np.array(imp_feats_count_list)
# # itemindex = np.where(imp_feats_count_arr in pos_top_imp_feats)
# print(pos_top_imp_feats)
# itemindex= np.searchsorted(imp_feats_count_arr, pos_top_imp_feats)
# itemindex

In [ ]:
# if isinstance(sel_feats, list):
#     print('sel_feats:',sel_feats)
#     mask = match_pos_df_actual['imp_feats_count'].isin(sel_feats)
#     match_pos_actual_filt_feats = match_pos_df_actual[mask]
# else:
#     match_pos_actual_filt_feats = match_pos_df_actual
    
# # match_pos_actual_filt_feats.head()

In [ ]:
# mask = (match_pos_actual_filt_feats['ratio_mean_asc']==sel_ratio_mean_asc) \
#         & (match_pos_actual_filt_feats['roll_mean_asc']==sel_roll_mean_asc)
# match_pos_actual_filt=match_pos_actual_filt_feats[mask]
# match_pos_actual_filt.sort_values('first_match_pos',ascending=True)

In [ ]:
# match_pos_actual_filt

In [ ]:
# # len(match_pos_df_actual)
# print('First Match Pos Mean:',match_pos_actual_filt_feats.groupby(['ratio_mean_asc','roll_mean_asc'])['first_match_pos'].mean().reset_index())

In [ ]:
# %%time
# # imp_feats_count_list = [2]
# imp_feats_count_list_cur = [sel_feats_count]

# # imp_feats_count_list= [2]+list(range(5,201,5))
# # imp_feats_count_list= list(range(2,151))

# sim_cutoff_dfs_actual=None
# top_n = len(all_k)
# cutoff_pos_df_actual,sim_cutoff_dfs_actual,top_mean_actual=create_cutoff_pos_df(test,xgb_models,
#                     features_test,df[df.index.isin(all_k)],imp_feats_count_list_cur,
#                     test_all_k,is_compute_matches=False,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
#                     low_high_record_condns=[(False,1,top_n)],
#                     ratio_mean_asc=sel_ratio_mean_asc,
#                     roll_mean_asc=sel_roll_mean_asc,
#                     top_k_ele=top_k_ele, is_print=True)
# cutoff_pos_df_actual

In [ ]:
# %%time
# k_list = top_mean_actual[0][:5]['k'].values
# test_sel=test_all_k[test_all_k['k'].isin(k_list)]
# test_sel[['k','roll_actual','client_seed','roll_0','roll_1']]

In [ ]:
# cols = ['hash_idx','k','imp_feats_40','imp_feats_35']
# col_to_sort = 'imp_feats_5'
# mask_temp = (k_order_df_actual['ratio_mean_asc']==0) & (k_order_df_actual['roll_mean_asc']==0) 
# k_order_df_filt_actual = k_order_df_actual[mask_temp]
# k_order_df_filt_actual = k_order_df_filt_actual.sort_values(col_to_sort)
# k_list = k_order_df_filt_actual['k'][:5].to_numpy()
# print(k_list)
# k_order_df_filt_actual[cols].head(5)

In [ ]:
# cols = ['hash_idx','k','imp_feats_10','imp_feats_35','imp_feats_40','ratio_mean_asc','roll_mean_asc']
# mask_temp = (k_order_df['ratio_mean_asc']==0) & (k_order_df['roll_mean_asc']==0) \
#             & (k_order_df['k'].isin(k_list)) \
#             & (k_order_df['imp_feats_40']<=100)

# k_order_df_filt = k_order_df[mask_temp]
# imp_feats_cols= [col for col in k_order_df_filt.columns if col.startswith('imp_feats')]
# k_order_df_filt['imp_mean']=k_order_df_filt[imp_feats_cols].mean(axis=1)
# k_order_df_filt = k_order_df_filt.sort_values(col_to_sort)
# k_order_df_filt[['imp_mean']+cols]


In [ ]:
# sel_imp_feats_top=[]
# for i in itemindex:
#     sel_imp_feats_top.extend(top_mean_actual[i][:5]['k'].values)
# sel_imp_feats_top=np.array(sel_imp_feats_top)
# np.array(np.unique(sel_imp_feats_top, return_counts=True)).T

In [ ]:
# %%time
# # imp_feats_count_list = [2,5,10,15,20,25,30,40]
# sim_cutoff_dfs_actual=None
# cutoff_pos_df_actual,sim_cutoff_dfs_actual,test_probs=create_cutoff_pos_df(test,xgb_models,
#                     features_test,df[df.index.isin(trans_k)],
#                     imp_feats_count_list,is_compute_matches=False,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
#                     low_high_record_condns=[(False,2,20)])
# cutoff_pos_df_actual

In [ ]:
# # k_sel,test_sel,cutoff_pos_best =get_test_sel_cutoff_pos(test_probs,cutoff_pos_df_hash,cutoff_pos_df_actual)
# # imp_feats_count_list = [2,5,10,15,20,25]

# k_sel,test_sel,top_mean_best =get_test_sel_cutoff_pos(test_probs,xgb_models,features_test,
#                                                       df,
#                             cutoff_pos_df_hash,
#                             is_min_total=False,total_cri=2,top_n=10,
#                             sim_cutoff_dfs=sim_cutoff_dfs_actual,
#                             imp_feats_count_list=imp_feats_count_list)
# print(f'{k_sel=}')
# print('cutoff best')
# print(top_mean_best)
# print(' ** selected test **')
# test_sel[['k','client_seed','probs']]

In [ ]:
# actual_seed="e1dbc502401294d4a38283e39d60e4d58cbaf5786f1b0770b3f6cf4f071bbeaa"
# #     actual_seed = cur_hash_list[25]
# print(actual_seed)
# # Vectorize the function
# vectorized_calculate_roll = np.vectorize(calculate_roll)

# # Compute the roll values for the input arrays
# roll_array = vectorized_calculate_roll(actual_seed,
#                                        test_all_k['client_seed'],
#                                        nonce)

# test_all_k['roll_actual_prev']=roll_array

In [ ]:
# %%time
# k_list = top_mean_actual[2][:5]['k'].values
# test_sel=test_all_k[test_all_k['k'].isin(k_list)]
# test_sel[['k','roll_actual_prev','roll_actual','client_seed','roll_0','roll_1']]

In [ ]:
# test_sel=None
# #array([ 10, 536, 371, 333, 518])
# k_list = top_mean_actual[37][:5]['k'].values
# print(k_list)
# for k_sel in k_list:
#     row = df.iloc[k_sel]
# #     print(row)
#     mask = (test['probs']>=row['cutoff']) & (test['probs']<=row['cutoff_2'])
#     cur_test_sel = test[mask]
#     cur_test_sel['k']=k_sel
#     if test_sel is None:
#         test_sel = cur_test_sel
#     else:
#         test_sel = pd.concat([test_sel,cur_test_sel],axis=0)
#     print('Test Size:',len(test_sel))
# test_sel[['k','roll_actual','client_seed','roll_0','roll_1']]

In [ ]:
# test_sel

END

In [ ]:
imp_feats = [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:10] 
test_sel[['k']+imp_feats]

In [ ]:
sim_hash_group[sim_hash_group['k']==24]

In [ ]:
sim_cutoff_df[sim_cutoff_df['k']==24]

In [ ]:
def compute_total(data,cutoffs,cols):
    mask = None
    for cutoff,col in zip(cutoffs,cols):
        cur_mask =(data[col]>=cutoff) 
        if mask is None:
            mask = cur_mask
        else:
            mask = mask & cur_mask
    data_filt = data[mask]
    total = len(data_filt)
    return total,data_filt

In [ ]:
cols=['roll_20_nonce','roll_14_nonce','roll_3_nonce','roll_16_nonce','roll_11_nonce']
cols=['roll_43_nonce','roll_56_nonce','roll_31_nonce','roll_20_nonce','roll_45_nonce']
cols=['roll_31_nonce','roll_52_nonce','roll_15_nonce','roll_45_nonce','roll_2_nonce']
cols=['roll_31_nonce','roll_52_nonce']
cols=['roll_52_nonce','roll_31_nonce','roll_24_nonce','roll_49_nonce','roll_41_nonce']
cols=['roll_mean_25_50_nonce','roll_52_nonce','roll_31_nonce']
cols=['roll_52_nonce','roll_31_nonce']
cols=['roll_46_nonce','roll_25_nonce']
cols=['roll_42_nonce','roll_37_nonce']

# cols=['roll_20_nonce','roll_15_nonce','roll_13_nonce','roll_12_nonce','roll_18_nonce']
# cols=['roll_20_nonce','roll_15_nonce','roll_13_nonce']
# cols=['roll_6_nonce','roll_11_nonce','roll_12_nonce','roll_10_nonce',]




In [ ]:
def get_cutoff(data,data_hash,col,quantile):
    mask=create_target_mask(data_hash)
#     cutoff = data[col].mean()-data_hash[col].mean()+data_hash[mask][col].mean()
#     cutoff = data_hash[mask][col].quantile(0.5)
    cutoff = data[col].mean()-data_hash[col].mean()+data_hash[mask][col].quantile(quantile)
    return cutoff



best_ratio =0
best_success = 0
best_total = 0
best_cutoffs = []
best_quantile = 0
best_actual = test_filt
# for quantile in np.linspace(0.01,0.95,50):
for quantile in np.linspace(0.01,0.95,50):
    cutoffs=[]
    for col in cols:
        #use entire hash to determine cutoff
        cutoff = get_cutoff(test,test_hash,col,quantile)
        cutoffs.append(cutoff)
    
#     print(cutoffs)
    total,filt_inter = compute_total(test_filt_hash,cutoffs,cols)
    mask2=create_target_mask(filt_inter)
#     mask2 = (filt_inter['roll_actual']>=9000)
    success = len(filt_inter[mask2])
    
    total_actual,filt_inter_actual = compute_total(test_filt,cutoffs,cols)
    mask2=create_target_mask(filt_inter_actual)

#     mask2 = (filt_inter_actual['roll_actual']>=9000)
    
    if total_actual==0:
        success_actual=0
        ratio_actual=0
    else:
        success_actual = len(filt_inter_actual[mask2])
        ratio_actual = success_actual / total_actual
    print('\nquantile:',quantile)
    print('Actuals:',ratio_actual,success_actual,total_actual)
    
    if total==0:
        ratio=0
    else:
        ratio = success / total
    print('Hash:',ratio,success,total)
#     print(quantile,cutoffs)
    print(total,total_actual)
#     print(total,success)
    if (total==0) or (success==0) : #or (total_actual<1):
        break

#     print(ratio,success,total)
    if ratio >= best_ratio:
        best_ratio = ratio
        best_success = success
        best_total = total
        best_cutoffs = cutoffs
        best_quantile = quantile
        best_actual = filt_inter_actual
print(best_quantile)        
print(best_ratio,best_success,best_total)        
print(best_cutoffs)
# print(test_filt[mask]['roll_actual'].describe())

In [ ]:
mask2=create_target_mask(best_actual)
success = len(best_actual[mask2])
total = len(best_actual)
ratio = success / total
print(ratio,success,total)

In [ ]:
test_sel = best_actual.copy()

In [ ]:


def get_actual_based_data(data,data_hash,data_filt,cols,total_cutoff=3):

    best_filt_inter = data
    best_total= 0
    best_quantile = 0
    for quantile in np.linspace(0.01,0.95,50):
        cutoffs=[]
        for col in cols:
            cutoff = get_cutoff(data,data_hash,col,quantile)
        #     cutoff = get_cutoff(test,test_hash,col)
            cutoffs.append(cutoff)
#         print(quantile,cutoffs)

        total,filt_inter=compute_total(data_filt,cutoffs,cols)
        if total<total_cutoff:
            break
        best_filt_inter = filt_inter
        best_total = total
        best_quantile = quantile
        
    return best_filt_inter,best_total,best_quantile

In [ ]:
for cutoff in range(0,11):

    best_filt_inter,best_total,best_quantile=get_actual_based_data(test,test_hash,test_filt,cols,
                                                                        total_cutoff=cutoff)
#     print('quantile:',best_quantile)
    mask2=create_target_mask(best_filt_inter)
    # mask2 = (best_filt_inter['roll_actual']>=9000)
    success = len(best_filt_inter[mask2])
    if best_total==0:
        ratio=0
        success=0
    else:
        success = len(best_filt_inter[mask2])
        ratio = success / best_total
    print(cutoff, ratio,success,best_total)
    # print(test_filt[mask]['roll_actual'].describe())

In [ ]:
test_sel =best_filt_inter.copy()

In [ ]:
best_filt_inter,best_total,best_quantile=get_actual_based_data(test_filt,test_filt_hash,test_filt,cols,
                                                              total_cutoff=4)
print('quantile:',best_quantile)
mask2=create_target_mask(best_filt_inter)
# mask2 = (best_filt_inter['roll_actual']>=9000)
success = len(best_filt_inter[mask2])
if best_total==0:
    ratio=0
    success=0
else:
    success = len(best_filt_inter[mask2])
    ratio = success / best_total
print(ratio,success,best_total)
# print(test_filt[mask]['roll_actual'].describe())

In [ ]:
# mask_actual = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
# idx= test_probs[mask_actual].index
# mask_hash = test_hash_probs.index.isin(idx)
# test_filt_hash_pos = test_hash_probs.loc[mask_hash]

mask_hash = (test_hash_probs['probs']>=row['cutoff']) & (test_hash_probs['probs']<=row['cutoff_2'])
idx= test_hash_probs[mask_hash].index
mask_actual = test_probs.index.isin(idx)
test_filt_pos = test_probs.loc[mask_actual]

best_filt_inter,best_total,best_quantile=get_actual_based_data(test,test_hash,
                                                               test_filt_pos,cols,
                                                              total_cutoff=37)
print('quantile:',best_quantile)
mask2=create_target_mask(best_filt_inter)
# mask2 = (best_filt_inter['roll_actual']>=9000)
if best_total==0:
    ratio=0
    success=0
else:
    success = len(best_filt_inter[mask2])
    ratio = success / best_total
print(ratio,success,best_total)
# print(test_filt[mask]['roll_actual'].describe())

In [ ]:
best_filt_inter

In [ ]:
test_sel=best_filt_inter.copy()

In [ ]:
# analysis_df_copy  = analysis_df.copy()

In [ ]:
def report_exceptional_teratio(analysis_df,digitlist,
                     ratio_cutoff,count_cutoff_min,count_cutoff_max,
                     pattern,bHighCheck,
                    zerocolcutoffs,is_ratio_cri_lessthan=False,
                               is_aftval_opp=False,is_aftval_sum=False):
    mask1 = analysis_df['digit'].isin(digitlist)
    if is_ratio_cri_lessthan:
        mask2= (analysis_df['tr_ratio']<ratio_cutoff) 
        if is_aftval_opp:
            mask2= mask2 & ((analysis_df['aft_all']>=ratio_cutoff) | (analysis_df['aft_1']>=ratio_cutoff) | (analysis_df['aft_2']>=ratio_cutoff)) 
    else:
        mask2=  (analysis_df['tr_ratio']>=ratio_cutoff) 
        if is_aftval_opp:
            mask2= mask2 & ((analysis_df['aft_all']<ratio_cutoff) | (analysis_df['aft_1']<ratio_cutoff) | (analysis_df['aft_2']<ratio_cutoff)) \
    
        
    mask= mask1 & mask2 \
             & ((analysis_df['aft_all']!=0) & (analysis_df['aft_1']!=0) & (analysis_df['aft_2']!=0)) \
                & (analysis_df['te_total']>=count_cutoff_min) \
                & (analysis_df['te_total']<=count_cutoff_max) \
                & (analysis_df['pattern_99']==pattern) \
                & (analysis_df['target_high']==bHighCheck) \
                & (analysis_df['zero_col_cutoff'].isin(zerocolcutoffs))
        
    if is_aftval_sum:
        mask= mask & ((analysis_df['aft_all']+analysis_df['aft_1']+analysis_df['aft_2'])>=36) \

    filtered = analysis_df[mask]
    
    records  = len(filtered)
    cols = ['tr_ratio','tr_total','te_total']
    
    return filtered

##### Production Code

In [ ]:
# def get_random_client_seed(test):
#     size = len(test)
#     test = test.reset_index(drop=True).sample(n=size).reset_index(drop=True)
#     rand_pos = random.randint(0,size-1)
#     print(f'rand_pos:{rand_pos}')
#     return test.iloc[rand_pos]['client_seed'],test

def get_random_client_seed(test):
    size = len(test)
    final_test= test.sample(n=size)
    rand_pos=random.randint(0,size-1)
    sample=final_test.reset_index(drop=True).loc[rand_pos]
#     rand_pos = sample.index[0]
    print(f'rand_pos:{rand_pos}')
    return sample['client_seed'],final_test

In [ ]:
client_seed,final_test = get_random_client_seed(test_sel)
print('Client Seed Selected')
print(client_seed)
# final_test.to_csv(f'data/analysis/final_test_{nonce}.csv')
# final_test.head()

In [ ]:
final_test

In [ ]:
# final_test.reset_index()[final_test.reset_index()['index']==6826]

After Actual SEED is known

In [ ]:
actual_seed="f9e1702bb3393f834aea8907e7e346bf9641e4c39131de555a19d56cf1b66989"
# actual_seed=cur_hash_list[4]
# print(f'{actual_seed=}')

In [ ]:
# Vectorize the function
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(actual_seed,
                                       final_test['client_seed'],
                                       nonce)
final_test['roll_actual_dummy']=final_test['roll_actual'].copy()
final_test['roll_actual']=roll_array
# final_test.to_csv(f'data/analysis/final_test_{nonce}.csv')

In [ ]:
final_test

In [ ]:
final_test.to_csv(f'data/final_test_{nonce}_{file_pattern_str}_pattern.csv')

In [ ]:
LOW_TARGET

In [ ]:
mask = create_target_mask(final_test)
success_pos = final_test[mask].index

if HIGH_ANALYSIS:
    success_pos = final_test[final_test['roll_actual']>=HIGH_TARGET].index
else:
    success_pos = final_test[final_test['roll_actual']<LOW_TARGET].index
print(len(success_pos))
print(f'success_pos:{success_pos}')

# target = set([17, 19, 34, 35, 44, 47])
# matched  = set(list(success_pos)).intersection(target)
# print(matched)

# print(len(matched)/len(success_pos))

In [ ]:
test_sel.head()

In [ ]:
final_test

In [ ]:
# Vectorize the function
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(actual_seed,
                                       results_df_test['client_seed'],
                                       nonce)

results_df_test['roll_actual']=roll_array
test_manual = generate_test_features(results_df_test)
test = generate_features_full(results_df_test,False,feature_chain_length)


analysis_df,test = gen_analysis(nonce,train_manual,test_manual,test,xgb_models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="")
analysis_df.head()

#### END

Finalize Training

In [ ]:
tr_index

In [ ]:
train.index

In [ ]:
# tr_index = train[30 * train_client_size:(30 + 5) * train_client_size].index
tr_index = train[train_client_size:].index
train_ignored = train[0:30*train_client_size]
# tr_index = list(tr_index) + list(train_ignored[train_ignored['target']==1].index)
print(len(tr_index))
xgb_model_final = xgb.XGBClassifier(**params)

X_tr,y_tr = X.iloc[tr_index],y.iloc[tr_index]
oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
# fit and apply the transform
X_tr, y_tr = oversample.fit_resample(X_tr, y_tr)
print(pd.Series(y_tr).value_counts())
xgb_model_final.fit(X_tr,y_tr)

In [ ]:
len(X_tr)

In [ ]:
test_probs = xgb_model_final.predict_proba(test)[:,1]

test_labels = convert_probtolabels(test_probs)
subm=pd.DataFrame()
subm['client_seed']= results_df_test['client_seed']
subm['preds'] = test_labels
subm['prob'] = test_probs
score = accuracy_score(y_test,test_labels)
print(f'test score:{score}')

In [ ]:
mask = (subm['prob']<0.52) & (subm['preds']==1)
print(len(subm[mask]))
score = accuracy_score(y_test[mask],test_labels[mask])
print(f'test score:{score}')